In [1]:
import gymnasium as gym
import os
#import gym
import numpy as np

import collections
import pickle
import tqdm

from stable_baselines3.common.buffers import ReplayBuffer

# Thoughts :

- so the degree of being a friend or a foe is
  a. degree of foe:  the amount of maximum possible value decressed only because of other players actions
      - over time...
      - the latest action.
  b. degree of friend: the amount of maximum possible value increase only because of the other players actions.

  This has be to taken at every valid action...right?


  C is an observer:
      A (-1|t)A(+1|t+1)A(+1|t+2)B(-1|t+3)B(-1|t+3)

      - then the Friend(A|C) =  the amount of V(C) increased by A's actions.
      - foe(A|C) = the amount of V(C) decreased by A's actions

- but there is a catch... the decision of friend and foe cannot be based on or should be conditioned on ...that friend did not make as self sacrifising dicisions.... i mean there could be actions which went wrong but had a different intentions....

- basically : P( actual intentions of of B | change is value of A  ) ... but how do we figure this out??

- okay ... so here is what I shall do ... i'll only identify how the sequential dilemma is handeled by the DT.

- but it would be better if we would identify and categorize the behaviour...should i use the change in value as an observation? ... but it would be redundant...


- can we promote relationship building ... and to be sly ? and will that really improve the chances to win?



- can it be a choice to maximize someone elses rewards? ... v(a') = v(a) + b_1.v(b) + b_2.v(c)

    - where b_1 and b_2 are actions? choice based on v(a') ... but estimating and maximizing v(a)
    - the idea is v(a) is proportional to v(a')
    - a positive b_1 is supporting ... a negative b_1 is definance.


Causality Confusion: The agent can't directly link its actions to the rewards because the environment changes independently. This makes it difficult to learn which actions are beneficial or harmful.


Reward Prediction Difficulty: Traditional reinforcement learning relies on predicting future rewards based on actions. In this case, the rewards are unpredictable and not directly tied to the agent's choices.

 - so the time other agents are taking actions.... as it is a sequential process.. the agent can model it as a markoves chains rather than markoves decisions process.... with Monte Carlo Simulation.

- I need to one hot encode the agent id.

- there is another way ... always quote the current agent as 1

- the states would be very similar but the reward function would change a lot .... how do we handle that : State Augmentation

# model arguments and model def.

In [1463]:
embe = nn.Embedding(10, 31)

In [1462]:
embedding = nn.Embedding(10, 3)
# a batch of 2 samples of 4 indices each
input_ = torch.LongTensor([[1, 2, 4, 5], [4, 3, 2, 9]])
embedding(input_)

tensor([[[-0.4950, -0.3356,  1.8569],
         [ 0.1044,  0.1453, -0.1328],
         [-0.6046, -0.1475,  0.1430],
         [ 0.0443,  1.2195,  0.6806]],

        [[-0.6046, -0.1475,  0.1430],
         [-0.1446,  1.0510, -1.0658],
         [ 0.1044,  0.1453, -0.1328],
         [-0.0913,  0.1200, -1.5220]]], grad_fn=<EmbeddingBackward0>)

In [1471]:
embe(torch.LongTensor([3]))

tensor([[ 1.1148,  2.2052,  0.3921,  0.6445, -0.0270,  1.6642,  0.8254,  0.2439,
         -0.4210,  0.8453,  2.0378, -0.7272, -0.5376, -0.4040,  1.1793, -0.5006,
          1.4405,  2.0716,  1.3809,  0.8354,  0.2485, -0.4491,  0.0299,  0.8506,
         -0.6823, -0.9633, -1.2667, -2.0696,  0.3557, -0.6044, -0.8769]],
       grad_fn=<EmbeddingBackward0>)

In [3]:
# docs and experiment results can be found at https://docs.cleanrl.dev/rl-algorithms/ppo/#ppopy
import os
import random
import time
from dataclasses import dataclass

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tyro
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter

from typing import Optional
@dataclass
class Args:
    exp_name: str = 'exp1_ddqn_'#os.path.basename(__file__)[: -len(".py")]
    """the name of this experiment"""
    seed: int = 1
    """seed of the experiment"""
    torch_deterministic: bool = True
    """if toggled, `torch.backends.cudnn.deterministic=False`"""
    cuda: bool = True
    """if toggled, cuda will be enabled by default"""
    track: bool = False
    """if toggled, this experiment will be tracked with Weights and Biases"""
    wandb_project_name: str = "cleanRL"
    """the wandb's project name"""
    wandb_entity: str = None
    """the entity (team) of wandb's project"""
    capture_video: bool = False
    """whether to capture videos of the agent performances (check out `videos` folder)"""
    TB_log:bool = True
    custom_env = True

    # Algorithm specific arguments
    env_id: str = "MountainCar-v0"#"CartPole-v1"
    """the id of the environment"""
    total_timesteps: int = 5000000
    """total timesteps of the experiments"""
    learning_rate: float = 3e-4#0.001#0.001 #2.5e-4
    """the learning rate of the optimizer"""
    num_envs: int = 1#4
    """the number of parallel game environments"""
    num_steps: int = 128
    """the number of steps to run in each environment per policy rollout"""
    anneal_lr: bool = True
    """Toggle learning rate annealing for policy and value networks"""
    gamma: float = 0.999 #0.99
    """the discount factor gamma"""
    gae_lambda: float = 0.98 #0.95
    """the lambda for the general advantage estimation"""
    num_minibatches: int = 4
    """the number of mini-batches"""
    update_epochs: int = 10#4
    """the K epochs to update the policy"""
    norm_adv: bool = True
    """Toggles advantages normalization"""
    clip_coef: float = 0.2
    """the surrogate clipping coefficient"""
    clip_vloss: bool = True
    """Toggles whether or not to use a clipped loss for the value function, as per the paper."""
    ent_coef: float = 0.01
    """coefficient of the entropy"""
    vf_coef: float = 0.5
    """coefficient of the value function"""
    max_grad_norm: float = 0.5
    """the maximum norm for the gradient clipping"""
    target_kl: float = None
    """the target KL divergence threshold"""


    learning_starts: int = 25e3
    tau: float = 0.005
    buffer_size: int = 10000
    # to be filled in runtime
    batch_size: int = 256
    """the batch size (computed in runtime)"""
    minibatch_size: int = 8
    """the mini-batch size (computed in runtime)"""
    num_iterations: int = 10
    """the number of iterations (computed in runtime)"""

#gae_lambda": 0.98, "ent_coef": 0.01, "vf_coef": 0.5, "max_grad_norm": 0.5, "batch_size": 64, "n_epochs": 4


def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer


class Agent(nn.Module):
    def __init__(self,envs,ob_space=None):
        if not ob_space:
            ob_space = envs.single_observation_space.shape
        
        
        super().__init__()
        self.critic = nn.Sequential(
            layer_init(nn.Linear(np.array(ob_space).prod(), 64)),
            nn.ReLU(),
            layer_init(nn.Linear(64, 64)),
            nn.ReLU(),
            layer_init(nn.Linear(64, 1), std=1.0),
        )
        self.actor = nn.Sequential(
            layer_init(nn.Linear(np.array(ob_space).prod(), 64)),
            nn.ReLU(),
            layer_init(nn.Linear(64, 64)),
            nn.ReLU(),
            layer_init(nn.Linear(64, envs.single_action_space.n), std=0.01),
        )

    def get_value(self, x):
        return self.critic(x)

    def get_action_and_value(self, x, action=None):
        logits = self.actor(x)
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.critic(x)
    def predict(self, x, action=None):
        return get_action_and_value(self, x, action=None)[0]



class Agent_shared(nn.Module):
    def __init__(self, envs):
        super(Agent, self).__init__()
        self.network = nn.Sequential(
            layer_init(nn.Linear(np.array(envs.single_observation_space.shape).prod(), 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 64)),
            nn.Tanh(),
        )
        self.actor = layer_init(nn.Linear(64, envs.single_action_space.n), std=0.01)
        self.critic = layer_init(nn.Linear(64, 1), std=1)

    def get_value(self, x):
        return self.critic(self.network(x))

    def get_action_and_value(self, x, action=None):
        hidden = self.network(x)
        logits = self.actor(hidden)
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.critic(hidden)
    def predict(self, x, action=None):
        return get_action_and_value(self, x, action=None)[0]



class Agent_shared_v1(nn.Module):
    def __init__(self, action_space = None, ob_space=None):
        super(Agent_shared_v1, self).__init__()
        self.network = nn.Sequential(
            layer_init(nn.Linear(ob_space, 64)),
            nn.GELU(),
            layer_init(nn.Linear(64, 64)),
            nn.GELU(),
        )
        self.actor = layer_init(nn.Linear(64, action_space), std=0.01)
        self.critic = layer_init(nn.Linear(64, 1), std=1)

    def get_value(self, x):
        return self.critic(self.network(x))

    def get_action_and_value(self, x, action=None):
        hidden = self.network(x)
        logits = self.actor(hidden)
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.critic(hidden)

    def predict(self, x, action=None):
        return get_action_and_value(self, x, action=None)[0]
    def get_valid_action(self,x):
        hidden = self.network(x)
        logits = self.actor(hidden)
        probs = Categorical(logits=logits)
        return probs.probs















class Agent_shared_v2(nn.Module):
    def __init__(self, action_space = None, ob_space=None):
                
        super().__init__()
        self.critic = nn.Sequential(
            layer_init(nn.Linear(ob_space, 64)),
            nn.ReLU(),
            layer_init(nn.Linear(64, 64)),
            nn.ReLU(),
            layer_init(nn.Linear(64, 1), std=1.0),
        )
        self.actor = nn.Sequential(
            layer_init(nn.Linear(ob_space, 64)),
            nn.ReLU(),
            layer_init(nn.Linear(64, 64)),
            nn.ReLU(),
            layer_init(nn.Linear(64, action_space), std=0.01),
        )

    def get_value(self, x):
        return self.critic(x)

    def get_action_and_value(self, x, action=None):
        logits = self.actor(x)
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.critic(x)
    def predict(self, x, action=None):
        return get_action_and_value(self, x, action=None)[0]





class Agent_shared_v1_risk(nn.Module):
    def __init__(self, action_space = None, ob_space=None):
        super(Agent_shared_v1_risk, self).__init__()
        self.network = nn.Sequential(
            layer_init(nn.Linear(ob_space, 64)),
            nn.GELU(),
            layer_init(nn.Linear(64, 64)),
            nn.GELU(),
            layer_init(nn.Linear(64, 64)),
            nn.GELU(),
            layer_init(nn.Linear(64, 64)),
            nn.GELU(),
        )
        self.actor_1 = layer_init(nn.Linear(64, action_space), std=0.01)
        self.actor_2 = nn.Sequential(layer_init(nn.Linear(64, 2), std=0.01), nn.Softmax(dim=1),)
        
        self.critic = layer_init(nn.Linear(64, 1), std=1)

    def get_value(self, x):
        return self.critic(self.network(x))

    def get_action_and_value(self, x, action=None,training=False):
        hidden = self.network(x)
        logits = self.actor_1(hidden)
        probs = Categorical(logits=logits)

        
        if action is None:
            action_1 = probs.sample()[:,None]
            action_2 = self.actor_2(hidden)[:,[0]]
    
            action = torch.concat((action_1,action_2),1)
        else:
            action_1,action_2 = action[:,0],action[:,1]

        if (len(x)>1) or (training):
            ret_p = probs.log_prob(action_1)*action_2
        else:
            ret_p = probs.log_prob(action_1[:,0])*action_2[:,0]

        
        
        return action, ret_p, probs.entropy(), self.critic(hidden)

    def predict(self, x, action=None):
        return get_action_and_value(self, x, action=None)[0]
    def get_valid_action(self,x):
        hidden = self.network(x)
        logits = self.actor(hidden)
        probs = Categorical(logits=logits)
        
        action_2 = self.actor_2(hidden)[:,[0]]
        
        return probs.probs,action_2





# ALGO LOGIC: initialize agent here:
class QNetwork(nn.Module):

    def __init__(self, action_space = None, ob_space=None):
        super().__init__()
        self.network = nn.Sequential(
            layer_init(nn.Linear(ob_space+action_space, 256)),
            nn.GELU(),
            #layer_init(nn.Linear(256, 256)),
            #nn.GELU(),
            layer_init(nn.Linear(256, 64)),
            nn.GELU(),
            layer_init(nn.Linear(64, 1)),
            nn.GELU(),
        )
        #self.actor_1 = layer_init(nn.Linear(64, action_space), std=0.01)
        #self.actor_2 = nn.Sequential(layer_init(nn.Linear(64, 2), std=0.01), nn.Softmax(dim=1),)
    def forward(self, x,a):
        x = torch.cat([x, a], 1)
        val = self.network(x)
        #hidden = self.network(x)
        #logits = self.actor_1(hidden)
        #probs = Categorical(logits=logits)

        #action_1 = probs.sample()[:,None]
        #action_2 = self.actor_2(hidden)[:,[0]]

        #action = torch.concat((action_1,action_2),1)

        return val #action




class Actor_ddqn(nn.Module):
    def __init__(self,env, action_space = None, ob_space=None):
        super().__init__()
        #self.fc1 = nn.Linear(ob_space, 256)
        #self.fc2 = nn.Linear(256, 256)
        #self.fc_mu = nn.Linear(256, action_space)


        self.network = nn.Sequential(
            layer_init(nn.Linear(ob_space, 256)),
            nn.GELU(),
            layer_init(nn.Linear(256, 256)),
            nn.GELU(),
            layer_init(nn.Linear(256, 64)),
            nn.GELU(),
            #layer_init(nn.Linear(64, 64)),
            #nn.GELU(),
        )
        self.actor_1 = layer_init(nn.Linear(64, action_space), std=0.01)
        self.actor_2 = nn.Sequential(layer_init(nn.Linear(64, 2), std=0.01), nn.Softmax(dim=1),)



        
        # action rescaling
        self.register_buffer(
            "action_scale", torch.tensor((env.action_space(1).high - env.action_space(1).low) / 2.0, dtype=torch.float32)
        )
        self.register_buffer(
            "action_bias", torch.tensor((env.action_space(1).high + env.action_space(1).low) / 2.0, dtype=torch.float32)
        )

    def forward(self, x):
        
        hidden = self.network(x)
        logits = self.actor_1(hidden)
        probs = Categorical(logits=logits)

        action_1 = probs.sample()[:,None]
        action_2 = self.actor_2(hidden)[:,[0]]

        action = torch.concat((action_1,action_2),1)

        return action#* self.action_scale + self.action_bias



    
#    def forward(self, x):
#        return self.network(x)


def linear_schedule(start_e: float, end_e: float, duration: int, t: int):
    slope = (end_e - start_e) / duration
    return max(slope * t + start_e, end_e)







In [79]:
#Agent_shared_v1_risk

# env model class 2

In [4]:
#env = gym.make(env_name,max_episode_steps=200)

env = gym.make('MountainCar-v0',max_episode_steps=2000)

class model_class_2(type(env)):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self.state_list = np.zeros((3,2))
        self.observation_space = gym.spaces.Box(low=-10 ,high=20, shape=(6,), dtype=np.float32)

    def step(self, action):
        
        observation, reward, terminated, truncated, info = self.env.step(action)
        self._elapsed_steps += 1

        if self._elapsed_steps >= self._max_episode_steps:
            truncated = True
        self.state_list[:2,:] = self.state_list[1:,:]
        self.state_list[2,:] = observation
        info['observation'] = observation.astype(np.float32)

        return self.state_list.reshape(-1), reward, terminated, truncated, info
        
    
    def reset(
            self,
            *,
            seed: Optional[int] = None,
            options: Optional[dict] = None,
            default = False,
            
            init_state=[-5,0]
                    ):
            super().reset(seed=seed)
            # Note that if you use custom reset bounds, it may lead to out-of-bound
            # state/observations.
            if default:
                low, high = utils_gym.maybe_parse_reset_bounds(options, -0.6, -0.4)
                self.state = np.array([self.np_random.uniform(low=low, high=high), 0])
            else:
                self.state = np.array(init_state,dtype='float32')
    
            if self.render_mode == "human":
                self.render()

            self.state_list = np.array([self.state],dtype=np.float32).repeat(3,axis =0)
            
            return self.state_list.reshape(-1), {'observation':self.state}
        


def make_env(env_id, idx, capture_video, run_name,custom_env=False,max_episode_steps=2000):
    def thunk():

                    
        if capture_video and idx == 0:
            env = gym.make(env_id, render_mode="rgb_array")
                
            env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        else:

            env = gym.make(env_id)

            
            #env = gym.make(env_id)
        if custom_env:
            #print('here')
            env = model_class_2(env=env,max_episode_steps=max_episode_steps)
        
        env = gym.wrappers.RecordEpisodeStatistics(env)
        return env

    return thunk




In [5]:
def expert_main():
    args = Args()#tyro.cli(Args)
    args.batch_size = int(args.num_envs * args.num_steps)
    args.minibatch_size = int(args.batch_size // args.num_minibatches)
    args.num_iterations = args.total_timesteps // args.batch_size
    run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{int(time.time())}"
    TB_log = args.TB_log
    if args.track:
        import wandb

        wandb.init(
            project=args.wandb_project_name,
            entity=args.wandb_entity,
            sync_tensorboard=True,
            config=vars(args),
            name=run_name,
            monitor_gym=True,
            save_code=True,
        )
    if TB_log:    
        writer = SummaryWriter(f"runs/{run_name}")
        writer.add_text(
            "hyperparameters",
            "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(args).items()])),
        )

    # TRY NOT TO MODIFY: seeding
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = args.torch_deterministic

    device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")

    # env setup
    envs = gym.vector.SyncVectorEnv(
        [make_env(args.env_id, i, args.capture_video, run_name,custom_env=args.custom_env) for i in range(args.num_envs)],
    )

    
    assert isinstance(envs.single_action_space, gym.spaces.Discrete), "only discrete action space is supported"

    ob_space_shape = envs.single_observation_space.shape#(3, 2)
    #ob_space = gym.spaces.Box(low=-10 ,high=20, shape=(3, 2), dtype=np.float32)
    #envs.single_observation_space = ob_space
    agent = Agent(envs,ob_space=None).to(device)
    optimizer = optim.Adam(agent.parameters(), lr=args.learning_rate, eps=1e-5)

    # ALGO Logic: Storage setup
    
    
    
    #change needed here
    obs = torch.zeros((args.num_steps, args.num_envs) + ob_space_shape).to(device)
    actions = torch.zeros((args.num_steps, args.num_envs) + envs.single_action_space.shape).to(device)
    logprobs = torch.zeros((args.num_steps, args.num_envs)).to(device)
    rewards = torch.zeros((args.num_steps, args.num_envs)).to(device)
    dones = torch.zeros((args.num_steps, args.num_envs)).to(device)
    values = torch.zeros((args.num_steps, args.num_envs)).to(device)

    # TRY NOT TO MODIFY: start the game
    global_step = 0
    start_time = time.time()
    next_obs, _ = envs.reset(seed=args.seed)
    next_obs = torch.Tensor(next_obs).to(device)
    next_done = torch.zeros(args.num_envs).to(device)

    for iteration in range(1, args.num_iterations + 1):
        # Annealing the rate if instructed to do so.
        if args.anneal_lr:
            frac = 1.0 - (iteration - 1.0) / args.num_iterations
            lrnow = frac * args.learning_rate
            optimizer.param_groups[0]["lr"] = lrnow

        for step in range(0, args.num_steps):
            global_step += args.num_envs
            obs[step] = next_obs
            dones[step] = next_done

            # ALGO LOGIC: action logic
            with torch.no_grad():
                #print(next_obs.shape)
                action, logprob, _, value = agent.get_action_and_value(next_obs)
                values[step] = value.flatten()
            actions[step] = action
            logprobs[step] = logprob

            # TRY NOT TO MODIFY: execute the game and log data.
            next_obs, reward, terminations, truncations, infos = envs.step(action.cpu().numpy())
            next_done = np.logical_or(terminations, truncations)
            rewards[step] = torch.tensor(reward).to(device).view(-1)
            next_obs, next_done = torch.Tensor(next_obs).to(device), torch.Tensor(next_done).to(device)

            if "final_info" in infos:
                for info in infos["final_info"]:
                    if info and "episode" in info:
                        if global_step%100 ==0:
                            print(f"global_step={global_step}, episodic_return={info['episode']['r']}")
                        if TB_log:  
                            writer.add_scalar("charts/episodic_return", info["episode"]["r"], global_step)
                            writer.add_scalar("charts/episodic_length", info["episode"]["l"], global_step)

        # bootstrap value if not done
        with torch.no_grad():
            next_value = agent.get_value(next_obs).reshape(1, -1)
            advantages = torch.zeros_like(rewards).to(device)
            lastgaelam = 0
            for t in reversed(range(args.num_steps)):
                if t == args.num_steps - 1:
                    nextnonterminal = 1.0 - next_done
                    nextvalues = next_value
                else:
                    nextnonterminal = 1.0 - dones[t + 1]
                    nextvalues = values[t + 1]
                delta = rewards[t] + args.gamma * nextvalues * nextnonterminal - values[t]
                advantages[t] = lastgaelam = delta + args.gamma * args.gae_lambda * nextnonterminal * lastgaelam
            returns = advantages + values

        # flatten the batch
        b_obs = obs.reshape((-1,) + envs.single_observation_space.shape)
        b_logprobs = logprobs.reshape(-1)
        b_actions = actions.reshape((-1,) + envs.single_action_space.shape)
        b_advantages = advantages.reshape(-1)
        b_returns = returns.reshape(-1)
        b_values = values.reshape(-1)

        # Optimizing the policy and value network
        b_inds = np.arange(args.batch_size)
        clipfracs = []
        for epoch in range(args.update_epochs):
            np.random.shuffle(b_inds)
            for start in range(0, args.batch_size, args.minibatch_size):
                end = start + args.minibatch_size
                mb_inds = b_inds[start:end]

                _, newlogprob, entropy, newvalue = agent.get_action_and_value(b_obs[mb_inds], b_actions.long()[mb_inds])
                logratio = newlogprob - b_logprobs[mb_inds]
                ratio = logratio.exp()

                with torch.no_grad():
                    # calculate approx_kl http://joschu.net/blog/kl-approx.html
                    old_approx_kl = (-logratio).mean()
                    approx_kl = ((ratio - 1) - logratio).mean()
                    clipfracs += [((ratio - 1.0).abs() > args.clip_coef).float().mean().item()]

                mb_advantages = b_advantages[mb_inds]
                if args.norm_adv:
                    mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)

                # Policy loss
                pg_loss1 = -mb_advantages * ratio
                pg_loss2 = -mb_advantages * torch.clamp(ratio, 1 - args.clip_coef, 1 + args.clip_coef)
                pg_loss = torch.max(pg_loss1, pg_loss2).mean()

                # Value loss
                newvalue = newvalue.view(-1)
                if args.clip_vloss:
                    v_loss_unclipped = (newvalue - b_returns[mb_inds]) ** 2
                    v_clipped = b_values[mb_inds] + torch.clamp(
                        newvalue - b_values[mb_inds],
                        -args.clip_coef,
                        args.clip_coef,
                    )
                    v_loss_clipped = (v_clipped - b_returns[mb_inds]) ** 2
                    v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
                    v_loss = 0.5 * v_loss_max.mean()
                else:
                    v_loss = 0.5 * ((newvalue - b_returns[mb_inds]) ** 2).mean()

                entropy_loss = entropy.mean()
                loss = pg_loss - args.ent_coef * entropy_loss + v_loss * args.vf_coef

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(agent.parameters(), args.max_grad_norm)
                optimizer.step()

            if args.target_kl is not None and approx_kl > args.target_kl:
                break

        y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy()
        var_y = np.var(y_true)
        explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y

        # TRY NOT TO MODIFY: record rewards for plotting purposes
        if TB_log:  
            writer.add_scalar("charts/learning_rate", optimizer.param_groups[0]["lr"], global_step)
            writer.add_scalar("losses/value_loss", v_loss.item(), global_step)
            writer.add_scalar("losses/policy_loss", pg_loss.item(), global_step)
            writer.add_scalar("losses/entropy", entropy_loss.item(), global_step)
            writer.add_scalar("losses/old_approx_kl", old_approx_kl.item(), global_step)
            writer.add_scalar("losses/approx_kl", approx_kl.item(), global_step)
            writer.add_scalar("losses/clipfrac", np.mean(clipfracs), global_step)
            writer.add_scalar("losses/explained_variance", explained_var, global_step)
        if global_step%100 ==0:
            print("SPS:", int(global_step / (time.time() - start_time)))
        if TB_log:  
            writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)

    envs.close()
    writer.close()
    return agent

# Risk env def

In [6]:
import functools
import random
from copy import copy

import numpy as np
from gymnasium.spaces import Discrete, MultiDiscrete, Box, Dict

from pettingzoo import AECEnv



import gymnasium


from pettingzoo.utils import agent_selector, wrappers

from gymnasium.utils import EzPickle



from statistics import NormalDist

import pygame

from typing import Any , Generic, Iterable, Iterator, TypeVar
ActionType = TypeVar("ActionType")

class agent_(np.int8):
    def bucket_set(self,val=10):
        self.bucket = val
    def add(self,val=0):
        self.bucket+=val
        return self
    def sub(self,val=0):
        self.bucket-=val
        return self



class Board:
    def __init__(self,default_attack_all = True,render_= False,agent_count = 3,env_=None,use_placement_perc = False,verbose = False):
        # internally self.board.squares holds a flat representation of tic tac toe board
        # where an empty board is [0, 0, 0, 0, 0, 0, 0, 0, 0]
        # where indexes are column wise order
        # 0 3 6
        # 1 4 7
        # 2 5 8

        # empty -- 0
        # player 0 -- 1
        # player 1 -- 2
        self.verbose = verbose
        self.env_ = env_
        self.agents = [agent_(0)] + [ agent_(i+1) for i in range(agent_count)]
        self.default_attack_all = default_attack_all
        self.render_ = render_
        self.unoccupied = agent_(0)
        self.unoccupied.bucket_set(0)
        self.reset_board()
        self.territory_count = len(self.territories)
        self.edge_count = len(self.edges)
        self.attack_dist_higher = NormalDist(mu=0, sigma=1)
        self.attack_dist_lower = NormalDist(mu=0, sigma=0.1)
        self.max_trials = 4
        self.max_bad_trials =4
        self.base_action_mask =  np.zeros(self.territory_count+self.edge_count +2, dtype='int8')

        self.use_placement_perc = use_placement_perc
        #send = new_troop_count//(1/action[1])
        
        
        #[0,1,2,3,4,5,6,7,8,9]
        self.territory_map = [[0,0],[0,1],
         [0,5],[0,6],
         [2,6],[2,5],[2,4],                
         [2,2],[2,1],[2,0]]
        
        
        self.colors_territory = [(255,255,0),(128,255,0),(0,128,255),(128,0,255),
                            (255,0,0),(255,128,0),(0,255,255),(0,0,255),
                            (255,0,255),(255,0,128)]
        
    
    def reset_board(self):
        self.continents  = np.empty(4, object)
        self.continents[:] = [[0,1],[2,3],[4,5,6],[7,8,9]]
        
        self.territories = np.zeros((10,2),dtype = object)


        #0  1   --   2  3
        #   |        |
        # 9 8 7 -- 6 5 4  

        self.territories[:,0] = self.unoccupied
        
        self.edges  = np.array([[0, 1],
                                 [1, 0],
                                 [1, 2],
                                 [1, 8],
                                 [2, 1],
                                 [2, 5],
                                 [2, 3],
                                 [3, 2],
                                 [4, 5],
                                 [5, 6],
                                 [5, 2],
                                 [5, 4],
                                 [6, 7],
                                 [6, 5],
                                 [7, 8],
                                 [7, 6],
                                 [8, 9],
                                 [8, 1],
                                 [8, 7],
                                 [9, 8]],dtype='int8')
        
    
        
        self.action_list = np.zeros((len(self.territories) + len(self.edges),2))

        self.current_agent = self.agents[1]
        self.agent_counter = 0
        self.phase = 0
        self.old_phase = -1
        self.bad_trials = 0

        #[i.bucket_set(int(i)*10) for i in self.agents]
        [i.bucket_set(3) for i in self.agents]

        self.territory_changes = np.zeros(len(self.agents))
        self.set_cycle()
        self.reset_screen()

        


    def reset_screen(self):
        if self.render_:
            
            pygame.init()
            self.screen = pygame.display.set_mode((7 * 125, 3* 125))
                
    def calculated_action_mask(self, agent_sel,phase = None):
        if type(phase) == type(None):
            phase = self.phase


        
            


        
        if phase == 0:
            
            action_mask =  np.array([1 if i in[agent_sel,0] else 0  for i in self.territories[:,0]]+[0]*self.edge_count +[1,1], dtype='int8')
        elif phase ==1:
            action_mask =  np.array([0]*self.territory_count+[ 1 if ((agent_sel == self.territories[i[0],0]) and (self.territories[i[0],1]>1)) and (agent_sel != self.territories[i[1],0]) else 0
                                                                  for i in self.edges] +[1,1], dtype='int8')

        else:
            action_mask =  np.array([0]*self.territory_count+[ 1 if ((agent_sel == self.territories[i[0],0] == self.territories[i[1],0]) and (self.territories[i[0],1]>1)) else 0
                                                                  for i in self.edges] +[0,1], dtype='int8')

        

        return action_mask
    
    def take_action(self, agent_sel,action,phase = None,mask = None):
        # if spot is empty
        #action[0] is action_index
            #phase 1 : action[1] # of new troops to be added
            #phase 2 : action[1] % maximum number of troops to be sent forward if attack succeeds. #this is kept to 1 by default.

        self.reset_reward()
        
        if type(phase) == type(None):
            phase = self.phase

        action[0] = np.int8(action[0])
        
        legal,end_turn = self.check_legal(agent_sel,action,phase,mask)
        if legal and not(end_turn):


            if phase ==0:

                if self.use_placement_perc:
                    
                    self.placement(agent_sel,action,convert_percent = self.use_placement_perc,troops=self.agents[agent_sel].bucket)
                else:
                    self.placement(agent_sel,action,convert_percent = self.use_placement_perc)
            
            elif phase == 1:
                self.attack(agent_sel,action)
            else:
                self.fortify(agent_sel,action)

                end_turn = 2
                

        
        if self.render_:
            self.render()

        self.ddnt_get_a_chance_to_place()

        return legal,end_turn, self.territory_changes
        
    def check_legal(self, agent_sel,action,phase = None,mask = None):

        agent = self.agents[agent_sel]
        
        if type(phase) == type(None):
            phase = self.phase

        
        if mask == None:
            mask = self.calculated_action_mask( agent_sel,phase)

        if mask[action[0]] ==0:
            if self.verbose :
                print(1)
            self.bad_trials +=1

            _,end_=self.check_bad_trials(base_end = 0)
            
            return False, end_ 
        # only changing unoccipied or self occupied territory and within bounds
        # for phase 0 : action only adds new troops to territory

        if action[0] == 31:
            return True,2
        elif action[0] == 30:
            return True,1
        else:
            if phase ==0:

                if self.use_placement_perc:
                    if (action[1] >1) or (action[1] <0):
                        if self.verbose :
                            print(2)
                        self.bad_trials +=1
                        _,end_=self.check_bad_trials(base_end = 0)
                        return False, end_
                    if agent.bucket == 0:
                        if self.verbose :
                            print(2.1)
                        self.bad_trials +=1
                        return False, 1
                    if (agent.bucket)*action[1] <1:
                        if self.verbose :
                            print(2.2)
                        self.bad_trials +=1
                        _,end_=self.check_bad_trials(base_end = 0)
                        return False, end_
                
                else:    
                
                    if action[1] <1:
                        if self.verbose :
                            print(2)
                        self.bad_trials +=1
                        _,end_=self.check_bad_trials(base_end = 0)
                        return False, end_
                    if agent.bucket == 0:
                        if self.verbose :
                            print(2.1)
                        self.bad_trials +=1
                        return False, 1
                    if agent.bucket < action[1]:
                        if self.verbose :
                            print(2.2)
                        self.bad_trials +=1
                        _,end_=self.check_bad_trials(base_end = 0)
                        return False, end_
                    
            elif phase ==3:
                if self.use_placement_perc:
                    if (action[1] >1) or (action[1] <0):
                        if self.verbose :
                            print(2)
                        self.bad_trials +=1
                        _,end_=self.check_bad_trials(base_end = 0)
                        return False, end_

                    edge = self.edges[action[0]-self.territory_count]
                    territory_a = self.territories[edge[0]]
                    
                    if (territory_a)*action[1] <1:
                        if self.verbose :
                            print(2.2)
                        self.bad_trials +=1
                        _,end_=self.check_bad_trials(base_end = 0)
                        return False, end_
                
                else:    
                
                    if action[1] <1:
                        if self.verbose :
                            print(2)
                        self.bad_trials +=1
                        _,end_=self.check_bad_trials(base_end = 0)
                        return False, end_
                    edge = self.edges[action[0]-self.territory_count]
                    territory_a = self.territories[edge[0]]
                    
                    if territory_a < action[1]:
                        if self.verbose :
                            print(2.2)
                        self.bad_trials +=1
                        _,end_=self.check_bad_trials(base_end = 0)
                        return False, end_                
            
            
            
            else:
                #phase 1 what percentage of troops to move forward. >=1
                if not(self.default_attack_all):
                    if action[1] >1:
                        if self.verbose :
                            print(3)
                        self.bad_trials +=1
                        _,end_=self.check_bad_trials(base_end = 0)
                        return False, end_
                    if self.territories[
                                    self.edges[action[0]-self.territory_count][0]
                                    ][1]//(1/action[1]) <1: #assuming nobody dies
                        if self.verbose :
                            print(4)
                        self.bad_trials +=1
                        _,end_=self.check_bad_trials(base_end = 0)
                        return False, end_
    
            if (self.current_agent == agent) :
                if phase != self.old_phase:
                    self.agent_counter =0 # meaning change in phase
                    self.old_phase = phase
                
                elif (self.agent_counter >= self.max_trials) and (phase == self.phase):
                    if self.verbose :
                        print(4.2)
                    
                    if self.phase ==0:
                        return False, 1
                    elif self.phase ==1:
                        return False, 1
                    else:
                        return False, 2
                        
                elif (self.agent_counter < self.max_trials) and (phase == self.phase):
                    self.agent_counter +=1
                    
    
                    #elif (phase == 2):
                    #    self.agent_counter = self.max_trials
                    #    return True, 2
            
            else:
                self.set_phase(phase)
                self.set_current_agent(agent)
                self.old_phase = phase
    
        
        return True,0
            
    def check_bad_trials(self,base_end = 0):
        if self.bad_trials >= self.max_bad_trials:
            if self.verbose :
                print(0.1)
            if self.phase ==0:
                return False, 1
            elif self.phase ==1:
                return False, 1
            else:
                return False, 2
                
        return False, base_end 
        

    def placement(self,agent_sel,action,convert_percent = False,troops=[],attack=False): #action = territory index, changein troops
        territory = self.territories[action[0]]

        if convert_percent:
            
            act_ = self.convert_percent_to_troops(troops,action[1],ceil=False)
        else:
            act_ = action[1]
        
        if self.verbose :
            print('curr_agent_board',self.current_agent,agent_sel,self.agents[agent_sel], 'act',act_ , 'target',action[0],self.territories[action[0]]    )
        if not(attack) and agent_sel !=self.unoccupied:
            self.agents[agent_sel].sub(act_)
        
        #print(self.agents[agent_sel],agent_sel)
        if self.territories[action[0]][0] ==0:
            self.territory_changes[agent_sel] += 1
        
        self.territories[action[0]] = [self.agents[agent_sel],territory[1]+act_]

    def fortify(self,agent_sel,action):


        edge = self.edges[action[0]-self.territory_count]
        
        territory_a = self.territories[edge[0]]
        territory_b = self.territories[edge[1]]

        agent = self.agents[agent_sel]
        
        troop_count = territory_a[1]
        
        if self.use_placement_perc:
            send = self.convert_percent_to_troops(troop_count,action[1],ceil=False,fortify=True)
            #send = new_troop_count//(1/action[1])
        else:
            send = min(troop_count,action[1]) - 1 #send all troops but 1

        
        self.placement(agent,[edge[0],-send],convert_percent = False,attack=True)
        self.placement(agent,[edge[1],send],convert_percent = False,attack=True)


    def set_phase(self,phase=0):
        self.old_phase = self.phase
        self.phase = phase
        self.bad_trials =0
        self.agent_counter =0
        
    def set_current_agent(self,agent):
        self.current_agent = agent
    def set_cycle(self,cycle=0):
        self.cycle=cycle
    def reset_reward(self):
        self.territory_changes = np.zeros(len(self.agents))

    def convert_percent_to_troops(self,troops,percent,ceil=False, fortify=False):

        if ceil and (percent >0.9):
            send = troops -1
        else:
            send = troops//(1/(percent+0.00000001))

            if (send == troops) and (fortify) : # when trying to send all the troops during fortification... you have to leave atleast 1
                send-=1
        return send

    
    def attack(self,agent_sel,action): # this does not care about mask.... run check_legal before attacking
        self.reset_reward()
        eta = 0.0000001
        #get edge
        #get troops on a and troops on b
        if self.verbose :
            print(0)
        edge = self.edges[action[0]-self.territory_count]
        
        territory_a = self.territories[edge[0]]
        territory_b = self.territories[edge[1]]
        
        agent_a = self.agents[territory_a[0]] 
        agent_b = self.agents[territory_b[0]] 
        
        # simplify - not roll, but just check if random is less than win_chance
        diff = (territory_a[1]- territory_b[1])

        win_chance = self.attack_dist_higher.cdf( diff/(territory_b[1]+eta)  ) if diff>0 else self.attack_dist_lower.cdf( diff/(territory_b[1] +eta) )
        if self.verbose :
            print(win_chance)

        if win_chance > np.random.rand():
            #opponent_territory becomes unoccupied, and troops become zero
            #winner looses same number of troops as opponent , self troops -1 , whichever is lesser
            

            #calculate total troops left , troops to retain at pos a and troops to send at pos b 
            loss = min(territory_b[1],territory_a[1]-1)
            new_troop_count = territory_a[1] - loss

            #print(loss,new_troop_count)
            
            
            send = 0
            
            if new_troop_count>1:
                if not(self.default_attack_all):
                    send = self.convert_percent_to_troops(new_troop_count,action[1],ceil=False)
                    #send = new_troop_count//(1/action[1])
                else:
                    send = new_troop_count - 1 #send all troops but 1
            
            
            
            if (send>0): # if we want to send then send, else just vacate the territory
                self.placement(agent_a,[edge[0],-loss-send],convert_percent = False,attack=True) #-

                if self.territories[edge[1]][0] !=0: #well it would be doublecounting othervise
                    self.territory_changes[agent_a] += 1
                    
                self.placement(agent_a,[edge[1],send],convert_percent = False,attack=True) #loose equal troops always


                
                self.territory_changes[agent_b] -= 1
            else:
                self.placement(agent_a,[edge[0],-loss],convert_percent = False,attack=True) #-
                self.placement(self.unoccupied,[edge[1],-territory_b[1]],convert_percent = False,attack=True) #-
                self.territory_changes[agent_b] -= 1
                
            if (agent_b != 0):
                if self.check_territory_count_N0(agent_b ):
                    if self.verbose :
                        print('here----')
                    if type(self.env_) != type(None):
                        self.env_.terminations[agent_b] =True
                        if self.verbose :
                            print('termination --',self.env_.terminations)
                        self.territory_changes[agent_a] += 1
                        self.territory_changes[agent_b] -= 100#50
        
        else:
            loss = min(territory_b[1]-1,territory_a[1]-1)
            self.placement(agent_a,[edge[0],-loss],convert_percent = False,attack=True) #-
            self.placement(agent_b,[edge[1],-loss],convert_percent = False,attack=True) #-

    def check_territory_count_N0(self,agent):
        k = sum(self.territories[:,0] == agent)
        if self.verbose :
            print('die?',agent,k)
        return (k ==0)

    def ddnt_get_a_chance_to_place(self):
        if 0 not in self.territories[:,0]:
            for i in self.env_.possible_agents:
                if not (self.env_.terminations[i]):
                    if i not in self.territories[:,0]:
                        self.env_.terminations[i] =True
                        if self.verbose :
                            print('termination --2',self.env_.terminations)
                        self.territory_changes[i] -= 100#50
                        
        
    def game_status(self):
        wins = np.array( np.unique(self.territories[:,0], return_counts=True)).T
        #all territory belong to a player
        #only one player lives and others dont have anything, its phase 1 and not the 1st cycle
        if len(wins)==1:
            status = wins[0][0]
        elif len(wins)==2 and self.cycle !=0 and (0 in wins[:,0]):
            status = wins[:,0][wins[:,0] !=0][0]
        else:
            status = 0
        return (status),wins

    # returns:
    # -1 for no winner
    # 1 -- agent 0 wins
    # 2 -- agent 1 wins
    def check_for_winner(self):
        pass

    def check_game_over(self,agent):
        pass

    def __str__(self):
        return (self.territories)


    def render(self,last_action_txt=''):
        #pygame.time.wait(500)
        self.screen.fill((255, 255, 255))  
        width = 125
        number_font = pygame.font.SysFont( None, 40 ) 
        number_font_2 = pygame.font.SysFont( None, 20 ) 
        for i,j in zip(self.territories,self.territory_map):
            cell_left = j[1]*width
            cell_top =  j[0]*width
            pygame.draw.rect(self.screen, self.colors_territory[i[0]], (cell_left, cell_top, width, width))
            
            # make the number from grid[row][col] into an image
            if i[0]:
                number_text = 'P'+str(i[0])+'-'+str(i[1])
            else:
                number_text = 'PFree'+'-'+str(i[1])
            
            #number_text  = f'P{ (i[0] if i[0] else 'Free') }-{i[1]}'
            number_image = number_font.render( number_text, True, (0,0,0), (255,255,255) )
        
            # centre the image in the cell by calculating the margin-distance
            # Note: there is no "height", cells are square (w by w)
            margin_x = ( width-1 - number_image.get_width() ) // 2
            margin_y = ( width-1 - number_image.get_height() ) // 2
        
            # Draw the number image
            self.screen.blit( number_image, ( cell_left+2 + margin_x, cell_top+2 + margin_y ) )

        # make the number from grid[row][col] into an image
        cell_left = 3*width
        cell_top =  1*width
        
        number_text  = "|".join([f'P{i}-: bucket {i.bucket}, T:{sum(self.territories[:,0]==i)} ' for i in self.agents[1:]])
        number_image = number_font_2.render( number_text, True, (0,0,0), (255,255,255) )
    
        # centre the image in the cell by calculating the margin-distance
        # Note: there is no "height", cells are square (w by w)
        margin_x = ( width-1 - number_image.get_width() ) // 2
        margin_y = ( width-1 - number_image.get_height() ) // 2

        # Draw the number image
        self.screen.blit( number_image, ( cell_left+2 + margin_x, cell_top+2 + margin_y ) )

        # make the number from grid[row][col] into an image
        cell_left = 3*width
        cell_top =  0.8*width
        
        number_text  = f'Phase :{self.phase}, cur_Player :{self.current_agent}, turn count:{self.agent_counter}, bad turns:{self.bad_trials},act{last_action_txt} '
        number_image = number_font_2.render( number_text, True, (0,0,0), (255,255,255) )
    
        # centre the image in the cell by calculating the margin-distance
        # Note: there is no "height", cells are square (w by w)
        margin_x = ( width-1 - number_image.get_width() ) // 2
        margin_y = ( width-1 - number_image.get_height() ) // 2

        # Draw the number image
        self.screen.blit( number_image, ( cell_left+2 + margin_x, cell_top+2 + margin_y ) )
            
        
        pygame.draw.line(self.screen,(0, 0, 0),[2*width,0.5*width],[5*width,0.5*width],1)
        pygame.draw.line(self.screen,(0, 0, 0),[1.5*width,1*width],[1.5*width,2*width],1)
        pygame.draw.line(self.screen,(0, 0, 0),[5.5*width,1*width],[5.5*width,2*width],1)
        pygame.draw.line(self.screen,(0, 0, 0),[3*width,2.5*width],[4*width,2.5*width],1)

        pygame.display.flip()
        pygame.display.update() 
        





class raw_env(AECEnv, EzPickle):
    metadata = {
        "render_modes": ["human", "rgb_array"],
        "name": "risk_tiny_v1",
        "is_parallelizable": False,
        "render_fps": 1,
    }

    def __init__(
        self, render_mode: str | None = None, screen_height: int | None = 400, default_attack_all : bool| None = False,
        render_:bool|None= False,agent_count : int|None = 3,use_placement_perc : bool|None = False, add_onturn : int | None =1
    , verbose:bool|None=False , bad_mov_penalization=0.1):
        super().__init__()
        EzPickle.__init__(self, render_mode, screen_height)
        self.verbose = verbose
        self.render_mode = render_mode
        self.screen_height = screen_height
        self.screen = None
        self.phases = [0,1,2]
        self.use_placement_perc = use_placement_perc
        self.add_onturn = add_onturn
        self.last_action_txt =''
        self.bad_mov_penalization = bad_mov_penalization
        
        if self.render_mode == "human":
            self.clock = pygame.time.Clock()

        
        
        self.board = Board(default_attack_all = default_attack_all, render_=render_,agent_count = agent_count,env_=self,use_placement_perc=use_placement_perc,
                          verbose = self.verbose)

        self.agents = self.board.agents[1:] #0th is unoccupied
        self.possible_agents = self.agents[:]
        self.last_agent = self.agents[0]


        if self.use_placement_perc:
            self.action_spaces = {j: {i: ( Box(low=np.array([0,0.1]), 
                                                 high=np.array([self.board.territory_count+ self.board.edge_count+2,
                                                                         1]), 
                                     shape=(2,), dtype=(np.float16)       
                                                            )
                                      if j == 0 
                                      else Box(low=np.array([self.board.territory_count,0]), 
                                                 high=np.array([self.board.territory_count + self.board.edge_count+2,
                                                                         1]), 
                                     shape=(2,), dtype=(np.float16)       
                                                            )
                                     )
                                      
                                      
                                      
                                      for i in self.agents}
                                for j in self.phases
                             }

        else:
        
            self.action_spaces = {j: {i: ( Box(low=np.array([0,0]), 
                                                 high=np.array([self.board.territory_count+ self.board.edge_count+2,
                                                                         10]), 
                                     shape=(2,), dtype=(np.int8)       
                                                            )
                                      if j == 0 
                                      else 

                                          ( Box(low=np.array([self.board.territory_count,0]), 
                                                 high=np.array([self.board.territory_count + self.board.edge_count+2,
                                                                         10]), shape=(2,), dtype=(np.int8)  )

                                          if j ==1
                                          
                                           else 
                                          
                                          Box(low=np.array([self.board.territory_count,0]), 
                                                 high=np.array([self.board.territory_count + self.board.edge_count+2,
                                                                         1]), shape=(2,), dtype=(np.float16)    )
                                     ))
                                      
                                      
                                      
                                      for i in self.agents }
                                for j in self.phases
                             }


                                
        
        self.observation_spaces = {
            i: Dict(
                {
                    "observation":  Box(low=np.zeros(self.board.territories.shape), 
                                    high=np.ones(self.board.territories.shape)*np.array([agent_count,100]), 
                                    shape=self.board.territories.shape, dtype=agent_
                    )
                                
                    
                    
                    ,
                    "action_mask": Box(low=0, high=1, shape=(self.board.territory_count + self.board.edge_count +2,), dtype=np.int8),
                }
                
            )
            for i in self.agents
        }

        self.rewards = {i: 0 for i in self.agents}
        self.terminations = {i: False for i in self.agents}
        self.truncations = {i: False for i in self.agents}
        #self.infos = {str(i): {} for i in self.agents}
        self.infos = {i: {} for i in self.agents}

        self.cycle = 0
        self._agent_selector = agent_selector(self.agents)

        self._phase_selector = agent_selector(self.phases)


        self.handle_change(next_cycle=0,reset_cycle=1,
                      next_agent=0,reset_agent=1,
                      next_phase=0,reset_phase=1)
        

    def observe(self, agent):
        
        #observation is the same for everyone atm
        observation = np.array(self.board.territories,dtype=np.int8)

        action_mask = np.array(self._get_mask(agent),dtype=np.int8)

        return {"observation": observation, "action_mask": action_mask}

    def _get_mask(self, agent):
        
        action_mask = np.zeros(self.board.territory_count + self.board.edge_count, dtype=np.int8)

        # Per the documentation, the mask of any agent other than the
        # currently selected one is all zeros.
        if agent == self.agent_selection:
            action_mask = self.board.calculated_action_mask(agent,phase = self.phase_selection)

        return action_mask

    #def _get_obs(self):
    #    return {"agent": self._agent_location, "target": self._target_location}

    #def _get_info(self):
    #    return {"distance": np.linalg.norm(self._agent_location - self._target_location, ord=1)}

    def observation_space(self, agent):
        
        return self.observation_spaces[agent]

    def action_space(self, agent):
        return self.action_spaces[self.phase_selection][agent]


    def reward_function(self,legal=True,end=0,territory_changes=[0,0,0,0],status=0,wins_=[[0,0],[0,0],[0,0],[0,0]]):

        if not(legal) and end:
            #print('subtracting')
            self.rewards[self.agents[self.agent_selection-1]] -= self.bad_mov_penalization
            self.curr_rewards[self.agents[self.agent_selection-1]] = -self.bad_mov_penalization
        elif (status != 0) and (self.all_deployed(wins_)): #game ended 
            if self.verbose :
                print('status',status)
            winner = status  # either TTT_PLAYER1_WIN or TTT_PLAYER2_WIN
            #loser = winner ^ 1  # 0 -> 1; 1 -> 0
            if self.verbose :
                print(winner)
            self.rewards[self.agents[winner-1]] += 100 #high reward forces the model to not just aqcuire territories, but also win
            self.curr_rewards[self.agents[winner-1]]=100
            for i in self.agents:
                try:
                    if i != winner:
                        self.rewards[self.agents[i-1]] -= 100
                        self.curr_rewards[self.agents[i-1]]=-100
                except Exception as e:
                    print(i,self.agents,self.rewards,winner)
                    raise Exception(e)  

            # once either play wins or there is a draw, game over, both players are done
            self.terminations = {i: True for i in self.agents}
            self._accumulate_rewards()
        else:
            for i,j in enumerate(territory_changes[1:]):
                
                #self.rewards[self.agents[i]]+=j
                self.curr_rewards[self.agents[i]]=j
                self.rewards[self.agents[i]]+=j
                

        #this is wrong ... only killed by some one elses actions
        #if (self.agent_selection not in wins_) and self.cycle>0:
        #    self.terminations[agent] =True
    #
    def handle_change(self,next_cycle=0,reset_cycle=0,
                      next_agent=0,reset_agent=0,
                      next_phase=0,reset_phase=0):
        if reset_cycle:
            self.cycle =0
            self.board.set_cycle( self.cycle)
        if next_cycle:
            self.cycle+=1
            self.board.set_cycle( self.cycle)
            
        if next_agent:
            
            if len(self._agent_selector.agent_order): # but this is not updating
                if self.verbose :
                    print('curr_agent_board',self.board.current_agent, 'curr_agent',self.agent_selection     )
                
                self.board.current_agent.add(self.add_onturn)
                self.agent_selection = self._agent_selector.next()
                while self.terminations[self.agent_selection] :
                    self.agent_selection = self._agent_selector.next()
                
                self.board.set_current_agent(self.agent_selection)
            
        if reset_agent:
            self.agent_selection = self._agent_selector.reset()
            while self.terminations[self.agent_selection] :
                    self.agent_selection = self._agent_selector.next()
            self.board.set_current_agent(self.agent_selection)
            
        if next_phase:
            self.phase_selection = self._phase_selector.next()
            
            self.board.set_phase(self.phase_selection)
        if reset_phase:
            self.phase_selection = self._phase_selector.reset()
            self.board.set_phase(self.phase_selection)
            

            
        
         

    # action in this case is a value from 0 to 8 indicating position to move on tictactoe board
    def step(self, action):
        #print()

        self.last_action_txt = ''
        if (
            self.terminations[self.agent_selection] or ( self.agent_selection in self.kill_list ) ):
            end = 3
        elif(sum(list(self.terminations.values()) ) >=2):
            end =4
        else:
            legal,end,territory_changes = self.board.take_action(self.agent_selection, action)

            #if ends and illegal... give a negative rewards?
            
            status,wins_ = self.board.game_status()
            self.reward_function(legal,end,territory_changes,status,wins_)
            self.last_action_txt = f'p{self.agent_selection}, action{action}'
  


        if self.verbose :
            print('live_list',self._agent_selector.agent_order,'phase',self._phase_selector.agent_order)
        
        
        
        not_removed = self.handle_post_cycle()#self.handle_terminations()
        
        
        if end==1:
            if self.verbose :
                print('1_')
            self.handle_change(next_cycle=0,reset_cycle=0,
                              next_agent=0,reset_agent=0,
                              next_phase=1,reset_phase=0)

        elif end==2:
            if self.verbose :
                print('2_')
            self.handle_change(next_cycle=1,reset_cycle=0,
                              next_agent=not_removed,reset_agent=0,
                              next_phase=0,reset_phase=1)
        elif end == 3:
            if self.verbose :
                print('3_')
            self.handle_change(next_cycle=0,reset_cycle=0,
                              next_agent=1,reset_agent=0,
                              next_phase=0,reset_phase=1)
            self.last_action_txt = f'p{self.agent_selection}, action 3_'
        elif end == 4:
            if self.verbose :
                print('3_')
            self.last_action_txt = f' P {self.agent_selection} WON!'
        
        if self.render_mode == "human":
            self.render(self.last_action_txt)

        
#######################

        

        #if self._deads_step_first():
        #self._accumulate_rewards()
        #else:
        #    self._clear_rewards()
        #    
        #if self._agent_selector.agent_order:
        #    self.agent_selection = self._agent_selector.next()

        #if self.env.frames >= self.env.max_cycles:
        #    self.terminations = dict(zip(self.agents, [True for _ in self.agents]))

        #self._cumulative_rewards[agent] = 0
        #self._accumulate_rewards()
        #self._deads_step_first()
        #self.steps += 1
##########################


        #observation = self.observe( self.agent_selection)#self._get_obs()
        #info = {observation['action_mask']}
        #observation = observation['observation']

        #return observation, reward, terminated, False, info




    

    def check_who_died(self):
        status,wins_ = self.board.game_status()
        if (self.all_deployed(wins_)):
            self.kill_list = list(set(self.agents ) - set(wins_[:,0]))
        else:
            self.kill_list = []
    
    def handle_post_cycle(self):
        not_removed = 1 
        if self._agent_selector.is_last(): #last person 2_
            self.check_who_died()
            iter_agents = self.agents[:]
            
            for agent in self.kill_list:#self.terminations:
                if ( agent in self._agent_selector.agent_order):
                    #print(agent,self.kill_list)
                    not_removed=0
                    self.terminations[agent]=True #if in killed list add to termination
                    iter_agents.remove(agent)
                    #self.agents.remove(agent)      #remove from list of alive agents

            if not(not_removed):
                self.kill_list = []
                
                self._agent_selector.reinit(iter_agents)
                #self._agent_selector.reinit(self.agents)  
                if self.verbose :
                    print('updated_order',self._agent_selector.agent_order)
        
        return 1#not_removed
        
    
    def handle_terminations(self):
        #is_last = self._agent_selector.is_last()
        not_removed = 1 
        
        iter_agents = self.agents[:]
        for agent in self.terminations:
            if (self.terminations[agent] or self.truncations[agent]) and ( agent in self._agent_selector.agent_order) :
                iter_agents.remove(agent)
                not_removed=0
                if self.verbose :
                    print('removed',agent)
        
        self.last_agent = self.agent_selection
        if not(not_removed):
            self._agent_selector.reinit(self.agents)
            if self.verbose :
                print('removed something',self._agent_selector.agent_order)
            
        
        return not_removed

    def all_deployed(self,wins):
        for i in self.agents:
            if not(self.init_deployment[i]) and ( i in wins[:,0]):
                self.init_deployment[i] =True

        return sum(self.init_deployment.values()) ==len(self.agents)


    
    def reset(self, seed=None, options=None):
        
        self.board.reset_board()
        self.curr_rewards = {i: 0 for i in self.agents}
        self.agents = self.possible_agents[:]
        self.rewards = {i: 0 for i in self.agents}
        self._cumulative_rewards = {i: 0 for i in self.agents}
        self.terminations = {i: False for i in self.agents}
        self.truncations = {i: False for i in self.agents}
        #self.infos = {str(i): {} for i in self.agents}
        self.infos = {i: {} for i in self.agents}
        # selects the first agent
        self._agent_selector.reinit(self.agents)

        
        self.handle_change(next_cycle=0,reset_cycle=1,
                              next_agent=0,reset_agent=1,
                              next_phase=0,reset_phase=1)
        
        self.kill_list = []

        if self.render_mode is not None and self.screen is None:
            pygame.init()

        if self.render_mode == "human":
            self.board.reset_screen()
            
            self.screen = self.board.screen
            pygame.display.set_caption("Tiny Risk")
        elif self.render_mode == "rgb_array":
            self.screen = pygame.Surface((400, 700))

        self.init_deployment = {agent:False for agent in self.agents}

        

        #observation = self._get_obs()
        #info = self._get_info()

        #if self.render_mode == "human":
        #    self._render_frame()

        #return observation, info






    

    def close(self):
        pass

    def render(self,last_action_txt=''):
        if self.render_mode is None:
            gymnasium.logger.warn(
                "You are calling render method without specifying any render mode."
            )
            return
        self.board.render(last_action_txt)

        #if self.render_mode == "human":
        
        self.clock.tick(self.metadata["render_fps"])
        observation = np.array(pygame.surfarray.pixels3d(self.screen))

        return (
            np.transpose(observation, axes=(1, 0, 2))
            if self.render_mode == "rgb_array"
            else None
        )




def env_risk(**kwargs):
    
    kwargs['render_mode'] = kwargs['render_mode'] if kwargs['render_mode'] != "ansi" else "human"
    env = raw_env(**kwargs)
    # This wrapper is only for environments which print results to the terminal
    if kwargs['render_mode'] == "ansi":
        env = wrappers.CaptureStdoutWrapper(env)
    #env = wrappers.TerminateIllegalWrapper(env, illegal_reward=-1)
    #env = wrappers.AssertOutOfBoundsWrapper(env)
    env = wrappers.OrderEnforcingWrapper(env)
    return env







env_config = dict(render_mode = 'rgb_array', default_attack_all  = True,
                render_ = True,agent_count  = 3,use_placement_perc=True)

env = env_risk(**env_config)



C:\Users\31721\.conda\envs\DL_project\lib\site-packages\pkg_resources\__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
C:\Users\31721\.conda\envs\DL_project\lib\site-packages\pkg_resources\__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
C:\Users\31721\.conda\envs\DL_project\lib\site-packages\pkg_resources\__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-

# training configurations

In [83]:
len(env.board.territories[:,0])

10

In [7]:
from pettingzoo.classic import  connect_four_v3 as game


#connect_four_v3

args = Args()#tyro.cli(Args)
args.batch_size = int(args.num_envs * args.num_steps)
args.minibatch_size = int(args.batch_size // args.num_minibatches)
args.num_iterations = args.total_timesteps // args.batch_size
run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{int(time.time())}"
TB_log = args.TB_log
if args.track:
    import wandb

    wandb.init(
        project=args.wandb_project_name,
        entity=args.wandb_entity,
        sync_tensorboard=True,
        config=vars(args),
        name=run_name,
        monitor_gym=True,
        save_code=True,
    )


TB_log = True
if TB_log:    
    writer = SummaryWriter(f"runs/{run_name}")
    writer.add_text(
        "hyperparameters",
        "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(args).items()])),
    )

# TRY NOT TO MODIFY: seeding
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.backends.cudnn.deterministic = args.torch_deterministic

device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")


In [8]:

playe_r = 1#"agent_1" #

num_steps = 1000000
action_shape = (2,)


env.reset(seed=42)

def obs_converter( data, num_classes = 4, col =0 ):

    if col != None:
        return torch.concat((nn.functional.one_hot(
                                                        data[:,col].detach().long(), 
                                                        num_classes = num_classes),
                                  data[:,~col,None]),axis=1)[:,1:]

def map_agent_phase_hot(data,num_classes = 3):
    return nn.functional.one_hot(torch.tensor(data),num_classes = num_classes)[1:]

def map_agent_phase_vector(data,num_classes = 3):
    return nn.functional.one_hot(data[:,0].long(), 
                                                        num_classes = num_classes)[:,1:]




sample_obs = obs_converter(torch.tensor(env.last()[0]['observation']),num_classes = 4)

ob_space_shape = sample_obs.shape #env.observation_space(playe_r)['observation'].shape
action_mask_shape = env.observation_space(playe_r)['action_mask'].shape

device= 'cpu'


In [9]:
qnet_config_dict = dict(action_space = env.action_space(playe_r).shape[0],
                            ob_space=np.prod(ob_space_shape)+np.prod(action_mask_shape)
                                 +1*( len(env.possible_agents) -1) #the current_agent +1#who actor agent was
                                 +1*(len(env.phases) -1)#the current phase
                                 +1 # the number of troops
                       )
actor_config_dict =  dict(env=env,action_space = env.observation_space(playe_r)['action_mask'].shape[0],
                            ob_space=np.prod(ob_space_shape)+np.prod(action_mask_shape)
                                 +1*( len(env.possible_agents) -1) #the current_agent +1#who actor agent was
                                 +1*(len(env.phases) -1)#the current phase
                                 +1 # the number of troops
                       )

In [361]:
qnet_config_dict

{'action_space': 2, 'ob_space': 77}

In [10]:
actor = Actor_ddqn(**actor_config_dict).to(device)
qf1 = QNetwork(**qnet_config_dict).to(device)
qf1_target = QNetwork(**qnet_config_dict).to(device)
target_actor = Actor_ddqn(**actor_config_dict).to(device)


target_actor.load_state_dict(actor.state_dict())
qf1_target.load_state_dict(qf1.state_dict())
q_optimizer = optim.Adam(list(qf1.parameters()), lr=args.learning_rate)
actor_optimizer = optim.Adam(list(actor.parameters()), lr=args.learning_rate)

In [11]:


rb = ReplayBuffer(
        args.buffer_size,
        Box(low =0, high=2000, shape =(qnet_config_dict['ob_space']+1,), dtype=int),
        Box(low =0, high=2000, shape =(2,), dtype=float),
        device,
        handle_timeout_termination=False,
    )
start_time = time.time()

In [12]:

#env = game.env(render_mode=None)#"human")



#agent_mod = Agent_shared_v1_risk(action_space = env.observation_space(playe_r)['action_mask'].shape[0],
#                            ob_space=np.prod(ob_space_shape)+np.prod(action_mask_shape)+1#the current_agent +1#who actor agent was
#                                 +1 #the current phase
#                                 +1 # the number of troops
#                                           )

#lr = 0.01
#optimizer = optim.AdamW(agent_mod.parameters(), lr=lr
#                        #args.learning_rate
#                       # , eps=1e-5
#                       )




obs = torch.zeros((num_steps,) + ob_space_shape).to(device)
actions = torch.zeros((num_steps, ) + action_shape).to(device)
action_masks = torch.zeros((num_steps, ) + action_mask_shape).to(device)


current_agent = torch.ones((num_steps,1)).to(device)*0#-1

current_phase = torch.zeros((num_steps,1)).to(device)

current_troops_count = torch.zeros((num_steps,len(env.possible_agents))).to(device)

logprobs = torch.zeros((num_steps, )).to(device)

rewards = torch.zeros((num_steps, len(env.possible_agents))).to(device)
rewards_2 = torch.zeros((num_steps, len(env.possible_agents))).to(device)
dones = torch.zeros((num_steps, len(env.possible_agents))).to(device)
values = torch.zeros((num_steps,  )).to(device)
episodes = torch.ones((num_steps, )).to(device)*-1

t_next = torch.zeros((num_steps, len(env.possible_agents))).to(device)



agent_list = list(env.agents)
global_step = 0
start_time = time.time()

faulting_player = ""

num_episodes = 100

phase = 0
args.minibatch_size = 128

In [365]:
#torchaudio==2.2.0
#torchdata==0.7.1
#torchtext==0.17.0
#torchvision==0.19.0

# model training

In [14]:
from IPython.display import clear_output, display

In [367]:
np.any( np.array([ curr_reward_list[i]  for i in env.agents if i != the_hero_agent]) == 100)

False

In [15]:
the_hero_agent = 1

num_episodes =5
env = env_risk(**(env_config | {"render_mode" : None, "bad_mov_penalization" : 0.01,"render_":False}))

args.gamma = gam = 0.99

gamma_t = {i:0 for i in env.possible_agents}

env.reset(42)

num_iterations = 1000

episode_time_lim = 5000



draw_count = 0

draw_territory_count = 0

first_count = 0
second_count = 0
third_count = 0

third_count_draw = 0





for iteration in range(1, num_iterations):#args.num_iterations) :
    #if args.anneal_lr:
    #        frac = 1.0 - (iteration - 1.0) / 10000#args.num_iterations
    #        lrnow = frac * lr#args.learning_rate
    #        optimizer.param_groups[0]["lr"] = lrnow


    step = 0
    fault_condition = False
    clear_output(wait=True)


    
    for episode in range(num_episodes):#num_episodes):
        
        total_rewards = {i:0 for i in env.possible_agents} #i can report this
        action=1
        if fault_condition:
            env = env_risk(**(env_config | {"render_mode" : None,"bad_mov_penalization" : 0.01,"render_":False}))#game.env(render_mode=None)
        env.reset()
        fault_condition = False
        #if step>0:
            #print( observation['action_mask'])
        

        step_count = 0

        bad_move_count = 0

        bad_move_phase_count = {i:0 for i in env.phases}
        move_count =  {i:0 for i in env.phases}


        is_draw = 0
        draw_territory_count = 0
        is_third = 0
        
        for agent in env.agent_iter():
            e_t = env.terminations
            if sum(e_t.values()) <2:
                observation, reward, termination, truncation, info = env.last()

                observation['observation'] =  obs_converter(torch.tensor(observation['observation'],device=device))
                
                episodes[step] = episode + (iteration-1)*num_episodes
                current_phase[step] = phase
                
                #global_step += args.num_envs
            
                if True:#action != None: # this is when a player is removed
                    
                    obs[step] = observation['observation']#torch.Tensor(observation['observation'],device=device) #sould i not add it .... meaning this is the last observation after the player dies
                    action_masks[step] = torch.Tensor(observation['action_mask'],device=device)
                    curr_agent = agent#int(agent[-1])
                    current_agent[step] = curr_agent
                    current_phase[step] = env.phase_selection
                    phase_mapping = map_agent_phase_hot(env.phase_selection,num_classes = len(env.phases))
                    curr_agent_mapping = map_agent_phase_hot(int(curr_agent)-1,num_classes = len(env.possible_agents))
                    
                    current_troops_count[step] = torch.Tensor([env.board.agents[i].bucket for i in env.possible_agents])
                    

                
                #obs[step] = next_obs
                #dones[step] = next_done
            
                #print(agent,action,termination or truncation, reward,info,observation['action_mask'],env.terminations,env.truncations)
        
        
        
        
        
        
                model_in = torch.Tensor(np.hstack((observation['observation'].reshape(-1),observation['action_mask'].reshape(-1),
                                                   phase_mapping,
                                                    curr_agent_mapping,
                                                   [env.board.agents[curr_agent].bucket ]))[None,:]#.repeat(3,axis = 0)
                                        )
        
                
                if termination or truncation:
                    action = None
                    with torch.no_grad():
                        act, logprob, _, value = agent_mod.get_action_and_value(model_in)
                        values[step] = value.flatten() # so even if we are removing the guy ... we need to know what is the action he would 
                                                        #have taken at this point and what would have been its value
                    actions[step] = act #even after going what would have been
                    logprobs[step] = logprob        
                else:
                    mask = observation["action_mask"]
                    # this is where you would insert your policy
                    #action = env.action_space(agent).sample(mask)
            
                    # ALGO LOGIC: action logic

                    #epsilon = linear_schedule(args.start_e, args.end_e, args.exploration_fraction * args.total_timesteps, global_step)
        
        
                    #if np.random.rand() > (episode + (iteration-1)*num_episodes)/(num_iterations*num_episodes):

                    if (global_step < args.learning_starts) or (
                        np.random.rand() > min(((episode + (iteration-1)*num_episodes)/((num_iterations*num_episodes)/10))
                            , 0.95)) or (agent != the_hero_agent):

                        
                        action = env.action_space(agent).sample()
                        part_0 =np.random.choice(np.where(env.board.calculated_action_mask(agent))[0])
                        action = torch.Tensor([[[part_0],[np.around(action[1],2)]]])
                        action = action[:,:,0]
                        #with torch.no_grad():
                            #print(next_obs.shape)
                            #value = q_network(torch.Tensor(model_in).to(device))
                            #action = actor(torch.Tensor(model_in).to(device))
                            #action += torch.normal(0, actor.action_scale * args.exploration_noise)
                            #action = actions.cpu().numpy().clip(env.action_space(agent).low, envs.action_space(agent).high)
                                
                            #action, logprob, _, value = agent_mod.get_action_and_value(model_in,action = action)
                            #action = action[:,:,0]
                            #actions[step] = action

                    else:
                        with torch.no_grad():
                            action = actor(torch.Tensor(model_in).to(device))
                        #action += torch.normal(0, actor.action_scale * args.exploration_noise)
                        #action = actions.cpu().numpy().clip(env.action_space(agent).low, env.action_space(agent).high)
                        #value = q_network(torch.Tensor(obs).to(device))
                        #action = torch.argmax(q_values, dim=1).cpu().numpy()
                                
                    #else:
                    
                    #    with torch.no_grad():
                            #print(next_obs.shape)
                            
                    #        action, logprob, _, value = agent_mod.get_action_and_value(model_in)
                    actions[step] = action
                    


                    if not observation['action_mask'][action[:,0].long()]: 
                        fault_condition =True
                        faulting_player = agent

                        if the_hero_agent == curr_agent:
                            bad_move_count+=1
                            bad_move_phase_count[int(current_phase[step][0])]+=1  # when is the where_is_it_performing_bad_really
                            #print('here',agent, action, observation['action_mask'])
                    

                    if the_hero_agent == curr_agent:
                        move_count[int(current_phase[step][0])]+=1
                    
                #print('here',agent, action)
                if action != None :
                    act_2 = action.detach().numpy()[0]#list([action.detach().numpy()[0][0], max(action.detach().numpy()[0][1],0.1) ])
                    act_2 = list([act_2[0], max(act_2[1],0.001) ])
                else:
                    act_2 = action
                    
                env.step(act_2 if action != None else None)
                


                if action == None:
                    rewards[step] = np.zeros(len(env.possible_agents)) # should i keep it -1? .... hm i dont think so .
                    dones[step] = np.zeros(len(env.possible_agents)) # frankly the guys is already done so we really dont have to do anything here.... this is the state post termination for a loser 
                    # but btw this is for the next agent ... action == None means in the last action the previous agent would have been removed.
                    #values[step] = 
                else:

                    rewards_2[step] = torch.Tensor([env.curr_rewards[i] for i in env.possible_agents])
                    curr_reward_list =  env.curr_rewards
                    if (step_count == (episode_time_lim-1)):
                        curr_reward_list = {i:-100 for i in env.possible_agents }

                    for i in env.possible_agents:
                        if i != curr_agent:
                            gamma_t[i]+=1
                        else:
                            gamma_t[i] =0



                        if (step_count == (episode_time_lim-1)):
                            cr_rew = -100
                            term = True
                        else:
                            cr_rew = env.curr_rewards[i]
                            term = env.terminations[i]
                        

                        

                        rewards[step-gamma_t[i],i-1] += (gam**gamma_t[i])*cr_rew
                        
                        t_next[step-gamma_t[i],i-1] = gamma_t[i]

                        
                        dones[step-gamma_t[i],i-1] = torch.Tensor([term]) #so the panetly has to be added but attributions is really difficult

                #list_curr_reward_list = np.array(list(curr_reward_list.values()))
                
                if sum(curr_reward_list.values()) == -300:
                    #print('here')
                    is_draw=1

                
                for age_i in env.possible_agents:
                    
                    total_rewards[age_i]+=curr_reward_list[age_i] #env.curr_rewards[age_i] if (step_count != episode_time_lim) else -100
                            
                    
                    #rewards[step] = torch.Tensor([env.curr_rewards[i] for i in env.possible_agents]) #reward
                    #dones[step] = torch.Tensor([env.terminations[i] for i in env.possible_agents ])#termination or truncation



                # so for the current actor gama_1 is probably 1 ... and remember to which back index to add.
                # maybe increase t, that is gamma^t * R[]
                # if the current actor, 


                
                step +=1
                global_step+=1

                





                
                #print(step-1,curr_agent,env.phase_selection,env.curr_rewards,rewards[step-1],action,gamma_t)
                #print('\naction',action,
                #      '\naction_valid',observation['action_mask'][int(action[0][0,None])],
                #      '\nagent',agent,
                #      '\nselected_agent',env.agent_selection,
                #      '\ncurr_agent',env.board.current_agent,
                #      '\ncurr phase',env.phase_selection,
                #      '\nbad_trail count',env.board.bad_trials,
                #      '\nmax_bad trails', env.board.max_bad_trials,
                #      '\nreward',reward,
                #      '\nreward',env.curr_rewards,
                #      '\nrewards',env.rewards,
                #      '\nbuckets', [env.board.agents[i].bucket for i in env.agents]
                #     )
                #print('kill_list',env.kill_list,'term',e_t, 'buckets',[i.bucket for i in env.board.agents])
            
            
            
            else:
                print('done:',env.terminations.values(),",total_reward:",total_rewards, ',iteration:',iteration,",episode:", episode )
                break    
        



            step_count+=1
            
            if (global_step == num_steps) :# or (fault_condition and (fa ulting_player != agent) and (len(env.agents)==0)):
                break
            elif (step_count == episode_time_lim):
                break
                
        #print(rewards[step-2])
        if global_step == num_steps:
            break


        position = 3
        for k_,(i_,j_) in enumerate(sorted([(j_,i_) for i_,j_ in total_rewards.items()],reverse=True) 
              ):
            if j_==the_hero_agent:
                position = k_+1
        #print(sum(curr_reward_list.values()),curr_reward_list.values())
        if is_draw:
            draw_count +=1
            if position ==3:
                third_count_draw +=1
                
            #    draw_territory_count = 0
            #else:
            draw_territory_count = int(observation['observation'][:,the_hero_agent].sum())

            writer.add_scalar("win_charts/3rd_position_all_prop",int(position ==3),((episode + (iteration-1)*num_episodes)+1))
            
            writer.add_scalar("win_charts/draw_count",draw_count,global_step)
            writer.add_scalar("win_charts/draw",1,((episode + (iteration-1)*num_episodes)+1))
            writer.add_scalar("win_charts/draw_to_total_count",draw_count/(episode + (iteration-1)*num_episodes +1),global_step)
            
            writer.add_scalar("win_charts/draw_territory_count",draw_territory_count,global_step)

        
            writer.add_scalar("win_charts/third_place_in_draw",third_count_draw,global_step)
            writer.add_scalar("win_charts/third_place_in_draw_ratio",third_count_draw/draw_count,global_step)
        else:
            
            first_count += (position == 1)
            second_count += (position == 2)
            third_count += (position == 3)
            writer.add_scalar("win_charts/1st_position_prop",int(position==1),((episode + (iteration-1)*num_episodes)+1))
            writer.add_scalar("win_charts/2nd_position_prop",int(position==2),((episode + (iteration-1)*num_episodes)+1))
            writer.add_scalar("win_charts/3rd_position_prop",int(position==3),((episode + (iteration-1)*num_episodes)+1))
            writer.add_scalar("win_charts/3rd_position_all_prop",int(position==3),((episode + (iteration-1)*num_episodes)+1))
            writer.add_scalar("win_charts/draw",0,((episode + (iteration-1)*num_episodes)+1))















            
            writer.add_scalar("win_charts/1st_position",first_count,((episode + (iteration-1)*num_episodes)+1))#global_step)

        
            writer.add_scalar("win_charts/2nd_position",second_count,((episode + (iteration-1)*num_episodes)+1))#global_step)
            writer.add_scalar("win_charts/3rd_position",third_count,((episode + (iteration-1)*num_episodes)+1))#global_step)  
            
            writer.add_scalar("win_charts/1st_position_to_total_terminated",first_count/((episode + (iteration-1)*num_episodes)-draw_count+1),((episode + (iteration-1)*num_episodes)+1))#global_step)

        
            writer.add_scalar("win_charts/2nd_position_to_total_terminated",second_count/((episode + (iteration-1)*num_episodes)-draw_count+1),((episode + (iteration-1)*num_episodes)+1))#global_step)
            writer.add_scalar("win_charts/3rd_position_to_total_terminated",third_count/((episode + (iteration-1)*num_episodes)-draw_count+1),((episode + (iteration-1)*num_episodes)+1))#global_step)             

        writer.add_scalar("win_charts/3rd_position_to_total",(third_count+third_count_draw)/(episode + (iteration-1)*num_episodes +1 ),((episode + (iteration-1)*num_episodes)+1))#global_step)

        
        writer.add_scalar("charts/epsilon",((episode + (iteration-1)*num_episodes)/((num_iterations*num_episodes)/10)),global_step)
        writer.add_scalar("charts/avg_per_epi_total_reward", np.mean(list(total_rewards.values())), global_step)
        writer.add_scalar("new_charts/bad_move_count_per_episode",bad_move_count,global_step)
        
        writer.add_scalar("new_charts/bad_move_count_position_per_episode",bad_move_phase_count[0],global_step)
        writer.add_scalar("new_charts/bad_move_count_attack_per_episode",bad_move_phase_count[1],global_step)
        writer.add_scalar("new_charts/bad_move_count_fortify_per_episode",bad_move_phase_count[2],global_step)

        writer.add_scalar("new_charts/total_moves",sum(move_count.values()),global_step)


        writer.add_scalar("new_charts/bad_move_to_step_count_per_episode",bad_move_count/(sum(move_count.values())+1),global_step)

        writer.add_scalar("new_charts/bad_move_to_step_position_per_episode",bad_move_phase_count[0]/( move_count[0]+1),global_step)
        writer.add_scalar("new_charts/bad_move_to_step_attack_per_episode",bad_move_phase_count[1]/( move_count[1]+1),global_step)
        writer.add_scalar("new_charts/bad_move_to_step_fortify_per_episode",bad_move_phase_count[2]/( move_count[2]+1),global_step)


        values_total = {i:0 for i in env.possible_agents}


        writer.add_scalar("charts/episodic_length", (episodes[:step] == (episode + (iteration-1)*num_episodes)).sum(), global_step)
        
        for i in env.possible_agents:
            cur_index = torch.where((current_agent[:,0] == i) &( episodes == (episode + (iteration-1)*num_episodes)  ))[0]

            #values_total[i] = values[cur_index].mean()
            #writer.add_scalar("charts/mean_value_per_epi_agent_"+str(i), values_total[i], global_step)
            
            writer.add_scalar("charts/total_reward_per_epi_agent_"+str(i), total_rewards[i], global_step)

    
        #writer.add_scalar("charts/avg_per_epi_value", np.mean(list(values_total.values())), global_step)

    # only learning from my experience

    #create_ a list of next observations
    # but only for hero agent.
    t_range = torch.Tensor(np.arange(0,step)).to(dtype=torch.int)
    hero_steps = [current_agent == the_hero_agent][0][:,0][:step]
    
    next_indecies = (t_next[:step,the_hero_agent-1].to(dtype=torch.int) + t_range +1)[hero_steps].long()

    selected_t_next = t_next[:,the_hero_agent-1,None]#t_next[:step,0,None][hero_steps]
    infos = [dir({})]*step #t_next[:step,0]
    b_obs_a = torch.concat(( obs.reshape(-1,np.prod(ob_space_shape)) ,
                            action_masks.reshape(-1,np.prod(action_mask_shape)),
                            map_agent_phase_vector(current_agent,num_classes=len(env.possible_agents)+1)[:,1:],
                            
                            #map_agent_phase_vector(current_agent,num_classes=len(env.possible_agents)),
                            map_agent_phase_vector(current_phase,num_classes=len(env.phases)),
                           current_troops_count[:,the_hero_agent-1,None],selected_t_next
                           ),axis =1)

    for i in zip(b_obs_a[:step][hero_steps], b_obs_a[next_indecies], 
       actions[:step][hero_steps], rewards[:step][hero_steps][:,the_hero_agent-1,None], 
       dones[:step][hero_steps][:,the_hero_agent-1,None], infos):
        rb.add(*i)


    for epoch in range(args.update_epochs):
        
        if global_step > args.learning_starts:
            data = rb.sample(args.batch_size)
            with torch.no_grad():
                collected_t_next = data.next_observations[:,-1]
                next_state_actions = target_actor(data.next_observations[:,:-1].to(dtype=torch.float))
                qf1_next_target = qf1_target(data.next_observations[:,:-1], next_state_actions)

                
                next_q_value = data.rewards.flatten() + (1 - data.dones.flatten()) * (args.gamma**(collected_t_next+1)).view(-1) * (qf1_next_target).view(-1)
        
            qf1_a_values = qf1(data.observations[:,:-1].to(dtype=torch.float32), data.actions.to(dtype=torch.float32)).view(-1)
            qf1_loss = nn.functional.mse_loss(qf1_a_values, next_q_value)
            
            # optimize the model
            q_optimizer.zero_grad()
            qf1_loss.backward()
            q_optimizer.step()
            
            #if global_step % args.policy_frequency == 0:
            actor_loss = -qf1(data.observations[:,:-1].to(dtype=torch.float32), actor(data.observations[:,:-1].to(dtype=torch.float32))).mean()
            actor_optimizer.zero_grad()
            actor_loss.backward()
            actor_optimizer.step()
        
            # update the target network
            for param, target_param in zip(actor.parameters(), target_actor.parameters()):
                target_param.data.copy_(args.tau * param.data + (1 - args.tau) * target_param.data)
            for param, target_param in zip(qf1.parameters(), qf1_target.parameters()):
                target_param.data.copy_(args.tau * param.data + (1 - args.tau) * target_param.data)

            ind_ = epoch + (iteration-1)*args.update_epochs
            
            writer.add_scalar("losses/qf1_values", qf1_a_values.mean().item(), ind_)
            
            writer.add_scalar("losses/qf1_loss", qf1_loss.item(), ind_)
            writer.add_scalar("losses/actor_loss", actor_loss.item(), ind_)
            #print("SPS:", int(global_step / (time.time() - start_time)))
            
    if global_step%100 ==0:
        print("SPS:", int(global_step / (time.time() - start_time)))       
    
    writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)

    avg_episode_length = np.mean([(episodes[:step] == i_epi).sum() for i_epi in episodes[:step].unique()])
    writer.add_scalar("charts/avg_episodic_length", avg_episode_length, global_step)

done: dict_values([True, True, True]) ,total_reward: {1: 105.63000000000008, 2: -200.01, 3: -109.36999999999999} ,iteration: 999 ,episode: 0
done: dict_values([True, True, True]) ,total_reward: {1: 103.31000000000004, 2: -200.16, 3: -112.66} ,iteration: 999 ,episode: 1
done: dict_values([True, True, True]) ,total_reward: {1: -200.70999999999998, 2: 106.86000000000003, 3: -98.49} ,iteration: 999 ,episode: 2
done: dict_values([True, True, True]) ,total_reward: {1: 105.79000000000005, 2: -106.24, 3: -200.03} ,iteration: 999 ,episode: 4


In [ ]:
env = env_risk(**(env_config | {"render_mode" : None,"verbose":False}))

for i__ in range(1):
    env.reset()#seed=42)
    #print(env_1.infos.keys())
    observation, reward, termination, truncation, info = env.last()
    
    total_rewards = {i:0 for i in env.possible_agents}
    
    #so there is an issue ...reward =2 sometimes ... doubling?

    for agent in env.agent_iter():
        e_t = env.terminations
        if sum(e_t.values()) <2:
            observation, reward, termination, truncation, info = env.last()
            
            action = env.action_space(agent).sample()
            
            #if env_1.phase_selection ==1:
            #    action = [int(action[0]),action[1]]
            part_0 =np.random.choice(np.where(env.board.calculated_action_mask(agent))[0])
            #print(part_0)
            
            #action = [part_0,np.around(action[1],2) #min(action[1],env_1.board.agents[agent].bucket ) 
            #                  if env_1.phase_selection==0 else  action[1]]
    
            action = [part_0,np.around(action[1],2)]
    
    
            for i in env.possible_agents:
                total_rewards[i]+=env.curr_rewards[i]
            
            env.step(action)
            print(env.board.territories)
            print('\naction',action,
                  '\naction_valid',observation['action_mask'][action[0]],
                  '\nagent',agent,
                  '\nselected_agent',env.agent_selection,
                  '\ncurr_agent',env.board.current_agent,
                  '\ncurr phase',env.phase_selection,
                  '\nbad_trail count',env.board.bad_trials,
                  '\nmax_bad trails', env.board.max_bad_trials,
                  '\nreward',reward,
                  '\nreward',env.curr_rewards,
                  '\nrewards',env.rewards,
                  '\nbuckets', [env.board.agents[i].bucket for i in env.agents]
                 )
            print('kill_list',env.kill_list,'term',e_t, 'buckets',[i.bucket for i in env.board.agents])
            #if sum(env.curr_rewards.values()) <0:
            #    input()
        else:
            print(env.board.territories)
            print('done',env.terminations.values(),env.curr_rewards)
            break

In [20]:
env.close()

In [913]:
map_agent_phase_vector(current_phase,num_classes=len(env.phases))

/var/folders/vl/9q_38sj961n1slqfmcpfjgnc0000gn/T/ipykernel_72257/1340412715.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return nn.functional.one_hot(torch.tensor(data[:,0].clone().detach()).long(),


tensor([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]])

In [912]:
map_agent_phase_vector(current_agent,num_classes=len(env.possible_agents)+1)[:,1:]

/var/folders/vl/9q_38sj961n1slqfmcpfjgnc0000gn/T/ipykernel_72257/1340412715.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return nn.functional.one_hot(torch.tensor(data[:,0].clone().detach()).long(),


tensor([[0, 0],
        [0, 0],
        [1, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]])

In [877]:
current_agent[:,0] == i

tensor([False, False, False,  ..., False, False, False])

In [876]:
current_agent[:100,0].unique()

tensor([1., 2., 3.])

In [875]:
nn.functional.one_hot(torch.tensor(current_agent[:100,0].clone().detach()+1).long(), 
                                                        num_classes = 3+1)[:,1:]

/var/folders/vl/9q_38sj961n1slqfmcpfjgnc0000gn/T/ipykernel_72257/2524108104.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  nn.functional.one_hot(torch.tensor(current_agent[:100,0].clone().detach()+1).long(),


RuntimeError: Class values must be smaller than num_classes.

In [870]:
current_agent.unique()
map_agent_phase_vector(current_agent[:4,],num_classes=len(env.possible_agents))

/var/folders/vl/9q_38sj961n1slqfmcpfjgnc0000gn/T/ipykernel_72257/4170305161.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return nn.functional.one_hot(torch.tensor(data[:,0].clone().detach()+1).long(),


RuntimeError: Class values must be smaller than num_classes.

In [851]:
map_agent_phase_vector(current_agent,num_classes=len(env.possible_agents))

/var/folders/vl/9q_38sj961n1slqfmcpfjgnc0000gn/T/ipykernel_72257/4170305161.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return nn.functional.one_hot(torch.tensor(data[:,0].clone().detach()+1).long(),


RuntimeError: Class values must be smaller than num_classes.

In [699]:
env.close()
writer.close()

In [ ]:
#torch.nn.functional.mse_loss.mse_loss

In [296]:
args.update_epochs

4

In [73]:

    




#env.close()
    with torch.no_grad():
        #advantages = torch.zeros_like(rewards).to(device)
    
        for i_epi in range((iteration-1)*num_episodes,episode + (iteration-1)*num_episodes +1):
            for agents_id in agent_list:
                agents_id_i = agents_id-1
                 
                cur_index = torch.where((current_agent[:,0] == agents_id) &( episodes == i_epi))[0]

                #values
            
                # bootstrap value if not done
            
                #next_value = agent.get_value(next_obs).reshape(1, -1)
                
                lastgaelam = 0
        
        
                last_index = cur_index[-1]
                nextnonterminal = 0 #1.0 it is the last and it is terminal ... so the next would be terminal too ... there should not be a next
                delta = rewards[last_index,agents_id_i] - values[last_index] # so even this should be find because ... rewards is already updated..
                #print(agents_id,last_index,nextnonterminal,delta,rewards[last_index],values[last_index])
                advantages[last_index] = lastgaelam = delta + args.gamma * args.gae_lambda * nextnonterminal * lastgaelam
                
                
                for o,t in reversed(list(enumerate(cur_index))[:-1]):
                    
        
                    nextnonterminal = 1.0 - dones[cur_index[o + 1],agents_id_i]
                    nextvalues = values[int(t  + t_next[t,agents_id_i] +1) ] #t  + t_next[t,agents_id]+1 is when the player acts next ...
                    delta = rewards[t,agents_id_i] + (args.gamma**t_next[t,agents_id_i]) * nextvalues * nextnonterminal - values[t] # t is the index when the player acted
                    #print(agents_id,o,t,cur_index[o + 1],nextnonterminal,nextvalues,delta,rewards[t])
                    advantages[t,agents_id_i] = lastgaelam = delta + args.gamma * args.gae_lambda * nextnonterminal * lastgaelam
        
        returns = advantages[range(advantages.shape[0]),current_agent[:,0].to(dtype=int)-1] + values
        
        
    #print(agent_list)
    b_obs = obs.reshape(obs.shape)
    b_logprobs = logprobs.reshape(-1)
    b_actions = actions.reshape(actions.shape)
    b_current_phase = current_phase.reshape(current_phase.shape)
    b_current_troops_count = current_troops_count[np.arange(len(current_troops_count)),current_agent[:,0].to(int)-1,None]
    b_action_masks = action_masks.reshape(action_masks.shape)
    b_advantages = advantages.reshape(-1)
    b_returns = returns.reshape(-1)
    b_values = values.reshape(-1)
    b_t_next =t_next.reshape(t_next.shape)
    b_obs_a = torch.concat(( b_obs.reshape(-1,np.prod(ob_space_shape)) ,
                            b_action_masks.reshape(-1,np.prod(action_mask_shape)),current_agent,current_phase,
                           b_current_troops_count
                           ),axis =1)

    # Optimizing the policy and value network
    b_inds = np.arange(step)#args.batch_size)
    clipfracs = []
    for epoch in range(args.update_epochs):
        np.random.shuffle(b_inds)
        for start in range(0, step, args.minibatch_size):
            end = start + args.minibatch_size
            mb_inds = b_inds[start:end]
    
            _, newlogprob, entropy, newvalue = agent_mod.get_action_and_value(b_obs_a[mb_inds],# b_obs[mb_inds].reshape(-1,np.prod(ob_space_shape)),#
                                                                              
                                                                          b_actions.long()[mb_inds],training=True)
            logratio = newlogprob - b_logprobs[mb_inds]
            ratio = logratio.exp()
    
            with torch.no_grad():
                # calculate approx_kl http://joschu.net/blog/kl-approx.html
                old_approx_kl = (-logratio).mean()
                approx_kl = ((ratio - 1) - logratio).mean()
                clipfracs += [((ratio - 1.0).abs() > args.clip_coef).float().mean().item()]
    
            mb_advantages = b_advantages[mb_inds]
            if args.norm_adv:
                mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)
    
            # Policy loss
            pg_loss1 = -mb_advantages * ratio
            pg_loss2 = -mb_advantages * torch.clamp(ratio, 1 - args.clip_coef, 1 + args.clip_coef)
            pg_loss = torch.max(pg_loss1, pg_loss2).mean()
    
            # Value loss
            newvalue = newvalue.view(-1)
            if args.clip_vloss:
                v_loss_unclipped = (newvalue - b_returns[mb_inds]) ** 2
                v_clipped = b_values[mb_inds] + torch.clamp(
                    newvalue - b_values[mb_inds],
                    -args.clip_coef,
                    args.clip_coef,
                )
                v_loss_clipped = (v_clipped - b_returns[mb_inds]) ** 2
                v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
                v_loss = 0.5 * v_loss_max.mean()
            else:
                v_loss = 0.5 * ((newvalue - b_returns[mb_inds]) ** 2).mean()
    
            entropy_loss = entropy.mean()
            loss = pg_loss - args.ent_coef * entropy_loss + v_loss * args.vf_coef
    
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(agent_mod.parameters(), args.max_grad_norm)
            optimizer.step()
    
        if args.target_kl is not None and approx_kl > args.target_kl:
            break

    y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy()
    var_y = np.var(y_true)
    explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y


    mean_value_per_episode_per_batch = np.mean([b_values[:step][(episodes[:step] == i_epi)].mean() for i_epi in episodes[:step].unique()])

    avg_episode_length = np.mean([(episodes[:step] == i_epi).sum() for i_epi in episodes[:step].unique()])
    
    # TRY NOT TO MODIFY: record rewards for plotting purposes
    if TB_log:

        writer.add_scalar("charts/mean_value_per_episode_per_batch", mean_value_per_episode_per_batch, global_step)
        writer.add_scalar("charts/learning_rate", optimizer.param_groups[0]["lr"], global_step)
        writer.add_scalar("charts/avg_episodic_length", avg_episode_length, global_step)

        
        writer.add_scalar("losses/value_loss", v_loss.item(), global_step)
        
        writer.add_scalar("losses/policy_loss", pg_loss.item(), global_step)
        writer.add_scalar("losses/entropy", entropy_loss.item(), global_step)
        writer.add_scalar("losses/old_approx_kl", old_approx_kl.item(), global_step)
        writer.add_scalar("losses/approx_kl", approx_kl.item(), global_step)
        writer.add_scalar("losses/clipfrac", np.mean(clipfracs), global_step)
        writer.add_scalar("losses/explained_variance", explained_var, global_step)
    if global_step%100 ==0:
        print("SPS:", int(global_step / (time.time() - start_time)))
    if TB_log:  
        writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)

env.close()
writer.close()    

            
#help if the player was killed before even his first play??? but that cannot happen .. to be killed we need to place something before hand

IndentationError: unexpected indent (385346547.py, line 2)

In [ ]:
avg_episode_length = np.mean([(episodes[:step] == i_epi).sum() for i_epi in episodes[:step].unique()])

In [2122]:
args.target_kl

In [2113]:
b_actions.long()[mb_inds].shape

torch.Size([1, 2])

In [2103]:
(current_agent[:,0] == agents_id)

( episodes == i_epi).sum()

tensor(250)

In [2102]:
torch.where((current_agent[:,0] == agents_id) &( episodes == i_epi))[0]

tensor([], dtype=torch.int64)

In [2097]:
episode + (iteration-1)*num_episodes

54

In [2095]:
episodes.unique()

tensor([-1., 38., 39., 47., 48., 49., 50., 51., 52., 53., 54.])

In [2093]:
torch.where((current_agent[:,0] == i) &( episodes == episode))[0]

tensor([], dtype=torch.int64)

In [530]:
oj = torch.tensor(obs[1][:,0])

/var/folders/vl/9q_38sj961n1slqfmcpfjgnc0000gn/T/ipykernel_72257/395297320.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  oj = torch.tensor(obs[1][:,0])


AttributeError: module 'numpy' has no attribute 'nunique'

In [541]:
#nn.functional.one_hot??

In [543]:
nn.functional.one_hot(oj.long(), num_classes = 4)


tensor([[0, 1, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0]])

In [2080]:
torch.concat(( b_obs.reshape(-1,np.prod(ob_space_shape)) ,
                         b_action_masks.reshape(-1,np.prod(action_mask_shape)),current_agent[:step],current_phase[:step],
                        b_current_troops_count
                        ),axis =1)


tensor([[ 0.,  0.,  0.,  ...,  1.,  0., 10.],
        [ 0.,  0.,  0.,  ...,  1.,  0.,  7.],
        [ 0.,  0.,  0.,  ...,  2.,  0., 20.],
        ...,
        [ 3.,  1.,  3.,  ...,  3.,  0.,  1.],
        [ 3.,  1.,  3.,  ...,  3.,  0.,  1.],
        [ 3.,  1.,  3.,  ...,  3.,  1.,  1.]])

In [2076]:
total_rewards

{1: 0, 2: 0, 3: 0}

In [2075]:
rewards[episodes == 90][:10]

tensor([[12.0000,  0.0000,  0.0000],
        [ 9.0000,  0.0000,  0.0000],
        [ 4.0100,  2.0000,  0.0000],
        [ 4.0000,  3.0000,  0.0000],
        [ 2.0000,  2.0000,  0.0000],
        [-1.0000,  3.0000,  0.0000],
        [-1.0000,  3.0000,  0.0000],
        [ 1.0000,  6.0000,  0.0000],
        [ 2.0000,  3.0000,  1.0000],
        [-2.9871,  4.0000,  1.0000]])

In [2060]:
sum(episodes == 94)

tensor(3479)

In [2063]:
([(i_epi,advantages[episodes == i_epi].sum(axis =0)) for i_epi in episodes[:step].unique()])

[(tensor(90.), tensor([ -10832.7793, -417176.3125, -269275.1250])),
 (tensor(91.), tensor([-105029.0078, -109808.8750,  -11357.9238])),
 (tensor(92.), tensor([-18237.9336, -17203.9180,  -1950.9956])),
 (tensor(93.), tensor([-18535.3398,  -7058.8452, -11997.2920])),
 (tensor(94.), tensor([-14566.3643,   -423.5226, -11490.5801]))]

In [2054]:

([(i_epi,rewards[episodes == i_epi].sum(axis =0)) for i_epi in episodes[:step].unique()])

[(tensor(90.), tensor([-12320.5859, -16273.6934,  -7868.3691])),
 (tensor(91.), tensor([-2840.3418, -3120.0928, -1878.1528])),
 (tensor(92.), tensor([-498.0001, -371.7140, -103.9209])),
 (tensor(93.), tensor([-506.3524, -160.7971, -258.6227])),
 (tensor(94.), tensor([-414.7703, -115.9055, -249.8394]))]

In [2046]:


np.mean([b_values[episodes == i_epi].mean() for i_epi in episodes[:step].unique()])

-3.3092284

In [1981]:
hid = agent_mod.network(b_obs_a[mb_inds]# b_obs[mb_inds].reshape(-1,np.prod(ob_space_shape)),#
                       )
actor_1_logit = agent_mod.actor_1(hid)
probs = Categorical(logits=actor_1_logit)

In [1987]:
action_1 = probs.sample()[:,None]

In [2001]:
action_2 = agent_mod.actor_2(hid)[:,[0]]

In [2002]:
action_2.shape

torch.Size([32, 1])

In [2011]:
torch.tensor([[1,2,3],[4,5,6],[7,8,9]])*torch.tensor([1,2,3])[:,None]

tensor([[ 1,  2,  3],
        [ 8, 10, 12],
        [21, 24, 27]])

In [2013]:
action_2

tensor([[0.4960],
        [0.4925],
        [0.4997],
        [0.4974],
        [0.4937],
        [0.4935],
        [0.5023],
        [0.4995],
        [0.4965],
        [0.4983],
        [0.4958],
        [0.4985],
        [0.4963],
        [0.4998],
        [0.4950],
        [0.4939],
        [0.4910],
        [0.5016],
        [0.4939],
        [0.4969],
        [0.4946],
        [0.4970],
        [0.4995],
        [0.4982],
        [0.4887],
        [0.4936],
        [0.5005],
        [0.4940],
        [0.4949],
        [0.4992],
        [0.4963],
        [0.4937]], grad_fn=<IndexBackward0>)

In [2015]:
probs.log_prob(action_1)*action_2

tensor([[-1.7124, -1.7190, -1.7144,  ..., -1.7168, -1.7008, -1.7128],
        [-1.7000, -1.6957, -1.6882,  ..., -1.6997, -1.6908, -1.7053],
        [-1.7295, -1.7229, -1.7181,  ..., -1.7278, -1.7381, -1.7397],
        ...,
        [-1.7263, -1.7301, -1.7307,  ..., -1.7252, -1.7279, -1.7288],
        [-1.7095, -1.7177, -1.7238,  ..., -1.7156, -1.7084, -1.7036],
        [-1.7304, -1.7374, -1.7339,  ..., -1.7244, -1.7234, -1.7417]],
       grad_fn=<MulBackward0>)

In [1989]:
probs.log_prob(action_1)

tensor([[-3.4521, -3.4654, -3.4562,  ..., -3.4611, -3.4287, -3.4528],
        [-3.4515, -3.4429, -3.4275,  ..., -3.4509, -3.4329, -3.4623],
        [-3.4613, -3.4480, -3.4385,  ..., -3.4579, -3.4784, -3.4817],
        ...,
        [-3.4580, -3.4656, -3.4668,  ..., -3.4558, -3.4612, -3.4630],
        [-3.4443, -3.4609, -3.4731,  ..., -3.4566, -3.4421, -3.4325],
        [-3.5051, -3.5191, -3.5121,  ..., -3.4928, -3.4909, -3.5278]],
       grad_fn=<SqueezeBackward1>)

In [ ]:
        probs = Categorical(logits=logits)

        
        if action is None:
            action_1 = probs.sample()[:,None]
            action_2 = self.actor_2(hidden)[:,[0]]
    
            action = torch.concat((action_1,action_2),1)
        else:
            action_1,action_2 = action[:,0],action[:,1]



        
    
        return action, probs.log_prob(action_1[:,0])*action_2[:,0], probs.entropy(), self.critic(hidden)

In [1978]:
actor_2 = agent_mod.actor_2(hid)

In [1980]:
*actor_2[:,0]

tensor([0.4960, 0.4925, 0.4997, 0.4974, 0.4937, 0.4935, 0.5023, 0.4995, 0.4965,
        0.4983, 0.4958, 0.4985, 0.4963, 0.4998, 0.4950, 0.4939, 0.4910, 0.5016,
        0.4939, 0.4969, 0.4946, 0.4970, 0.4995, 0.4982, 0.4887, 0.4936, 0.5005,
        0.4940, 0.4949, 0.4992, 0.4963, 0.4937], grad_fn=<SelectBackward0>)

In [1943]:
logprobs.shape

torch.Size([1000000])

In [1948]:
current_agent.shape

torch.Size([1000000, 1])

In [1953]:
current_phase
current_troops_count.shape

torch.Size([1000000, 1])

In [ ]:
[curr_agent,env.phase_selection,env.board.agents[curr_agent].bucket ]

In [1965]:
t_next

tensor([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  9., 11.]])

In [1967]:
        
#print(agent_list)
b_obs = obs.reshape(obs.shape)
b_logprobs = logprobs.reshape(-1)
b_actions = actions.reshape(actions.shape)
b_current_phase = current_phase.reshape(current_phase.shape)
b_current_troops_count = current_troops_count[np.arange(len(current_troops_count)),current_agent[:,0].to(int)-1,None]
b_action_masks = action_masks.reshape(action_masks.shape)
b_advantages = advantages.reshape(-1)
b_returns = returns.reshape(-1)
b_values = values.reshape(-1)
b_t_next =t_next.reshape(t_next.shape)
b_obs_a = torch.concat(( b_obs.reshape(-1,np.prod(ob_space_shape)) ,
                        b_action_masks.reshape(-1,np.prod(action_mask_shape)),current_agent,current_phase,
                       b_current_troops_count
                       ),axis =1)


every_person must only look at its own troops at the time ... so we dont need to look at what happend to my troop when its not my turn

In [1964]:
current_phase.shape

torch.Size([1000000, 1])

In [1962]:
current_troops_count[np.arange(len(current_troops_count)),current_agent[:,0].to(int)-1,None]

tensor([[10.],
        [ 7.],
        [ 7.],
        ...,
        [ 0.],
        [ 0.],
        [ 0.]])

In [ ]:
current_troops_count

In [1970]:
(obs.shape
,logprobs.shape
,actions.shape
,current_phase.shape
,current_troops_count.shape
,action_masks.shape
,advantages.shape
,returns.shape
,values.shape
)

(torch.Size([1000000, 10, 2]),
 torch.Size([1000000]),
 torch.Size([1000000, 2]),
 torch.Size([1000000, 1]),
 torch.Size([1000000, 3]),
 torch.Size([1000000, 32]),
 torch.Size([1000000, 3]),
 torch.Size([1000000]),
 torch.Size([1000000]))

In [1811]:
torch.where((current_agent[:,0] == 1) &( episodes == 0))[0]

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,  22,  23,  24,  25,  26,
         35,  36,  37,  38,  44,  51,  52,  53,  54,  55,  70,  73,  74,  75,
         76,  77,  78,  79,  80,  81,  90,  96, 105, 106, 118, 125, 126, 127,
        128, 134, 143, 144, 145, 146, 147, 148, 149, 157, 158, 165, 178, 193,
        194, 195, 196, 197, 205, 206, 207, 208, 209, 210, 221, 229, 230, 233,
        234, 235, 240, 241, 242, 243, 251, 252, 254, 262, 268, 269, 275, 276,
        277, 278])

In [1815]:
torch.where((current_agent[:,0] == 3) &( episodes == 0))[0]

tensor([ 13,  14,  15,  16,  17,  18,  19,  20,  21,  31,  32,  33,  34,  41,
         42,  43,  47,  48,  49,  50,  62,  63,  64,  65,  66,  67,  68,  69,
         72,  87,  88,  89,  92,  93,  94,  95, 100, 101, 102, 103, 104, 116,
        117, 124, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 154, 155,
        156, 161, 162, 163, 164, 173, 174, 175, 176, 177, 185, 186, 187, 188,
        189, 190, 191, 192, 199, 200, 201, 202, 203, 204, 213, 214, 215, 216,
        217, 218, 219, 220, 222, 223, 224, 225, 226, 227, 228, 231, 232, 236,
        237, 238, 239, 244, 245, 246, 247, 248, 249, 250, 253, 255, 256, 257,
        258, 259, 260, 261, 263, 264, 265, 266, 267, 270, 271, 272, 273, 274,
        279, 280, 281, 282, 283, 284, 285, 286])

In [1817]:
values[torch.where((current_agent[:,0] == 3) &( episodes == 0))[0]]

tensor([-4.5665, -4.5665, -3.2696, -0.3031,  1.1337,  1.1337,  1.1337,  1.2421,
         1.3567,  1.1014,  1.1014,  1.1014,  1.1014,  1.0461,  1.1337,  1.2421,
         1.1014,  1.1014,  1.1014,  1.1337,  1.1014,  1.1014,  1.2041,  1.2041,
         1.2041,  1.2041,  1.2041,  1.2041,  1.0461,  0.9699,  1.0692,  1.0692,
         0.8751,  1.1436,  1.2592,  1.3845,  1.1337,  1.1337,  1.1337,  1.2421,
         1.2421,  1.1014,  1.1014,  1.0461,  0.9699,  0.9699,  1.0461,  0.9699,
         1.1337,  1.1337,  1.1337,  1.1337,  1.2421,  1.3567,  1.1014,  1.1014,
         1.1014,  1.0461,  1.1014,  1.1014,  1.1436,  1.1337,  1.1337,  1.1337,
         1.1337,  1.2421,  1.1014,  1.1337,  1.1337,  1.1337,  1.2421,  1.2421,
         1.2421,  1.3567,  1.1014,  1.1014,  1.1337,  1.1337,  1.1337,  1.2421,
         1.1014,  1.2041,  1.2041,  1.2041,  1.2041,  1.2041,  1.2041,  1.3094,
         1.0461,  1.1337,  1.1337,  1.1337,  1.1337,  1.2421,  1.3567,  1.1014,
         1.1337,  1.1014,  1.1337,  1.24

In [1819]:
values[torch.where((current_agent[:,0] == 2) &( episodes == 0))[0]]

tensor([-2.5403,  0.5215,  0.6688,  0.9050,  0.7962,  1.0707,  1.0707,  1.1576,
         1.1251,  1.1251,  1.0707,  1.1408,  0.9966,  0.9966,  0.9966,  1.1576,
         1.1576,  1.2425,  1.1251,  1.0707,  1.0707,  1.1251,  1.2014,  1.2014,
         1.0707,  0.9966,  1.0639,  1.0639,  0.9050,  1.1576,  1.1576,  1.1576,
         1.1576,  1.2425,  1.2425,  1.2425,  1.3316,  1.1251,  1.1251,  1.2014,
         1.2802,  1.2802,  1.0707,  1.0707,  0.9966,  0.9050,  0.9731,  0.9731,
         0.9731,  0.7962,  1.0707,  0.9966,  1.1576,  1.1576,  1.1576,  1.1576,
         1.2425,  1.3316,  1.1251,  1.1251,  1.1576,  1.1576,  1.1576,  1.2425,
         1.1251,  1.0707,  1.0707])

In [1812]:
values[torch.where((current_agent[:,0] == 1) &( episodes == 0))[0]]

tensor([0.2430, 1.1102, 1.1102, 1.1885, 1.1885, 1.1885, 1.2440, 1.2440, 1.2440,
        1.1593, 1.1885, 1.1885, 1.1885, 1.1885, 1.1593, 1.1885, 1.2440, 1.3055,
        1.1593, 1.1102, 1.1593, 1.2056, 1.2056, 1.2575, 1.1102, 1.0430, 1.0430,
        1.0430, 1.1102, 1.1885, 1.1885, 1.2440, 1.2440, 1.3055, 1.1593, 1.1102,
        1.0430, 1.0430, 0.9588, 0.8571, 0.8571, 0.8944, 0.9262, 0.7360, 0.5937,
        1.1102, 1.1885, 1.2440, 1.2440, 1.2440, 1.3055, 1.1593, 1.1885, 1.1593,
        1.1102, 1.0430, 1.1102, 1.1593, 1.1593, 1.1885, 1.1593, 1.1885, 1.1885,
        1.2440, 1.2440, 1.3055, 1.1593, 1.1102, 1.1102, 1.0430, 1.0430, 1.0430,
        0.9588, 0.9941, 0.9941, 1.0273, 0.8571, 0.8571, 0.7360, 0.5937, 0.4294,
        0.4785, 0.2430, 0.2916, 0.2916, 0.2916])

In [1863]:
advantages.sum()

tensor(0.)

In [1862]:
for i in range(step):
    if not(all(rewards[i] == torch.zeros(3))):
        print(i,rewards[i],t_next[i],rewards_2[i],dones[i],advantages[i])

0 tensor([1., 0., 0.]) tensor([0., 0., 0.]) tensor([1., 0., 0.]) tensor([0., 0., 0.]) tensor([0., 0., 0.])
2 tensor([1., 0., 0.]) tensor([0., 0., 0.]) tensor([1., 0., 0.]) tensor([0., 0., 0.]) tensor([0., 0., 0.])
5 tensor([-1.,  0.,  0.]) tensor([0., 0., 0.]) tensor([-1.,  0.,  0.]) tensor([0., 0., 0.]) tensor([0., 0., 0.])
6 tensor([1., 0., 0.]) tensor([0., 0., 0.]) tensor([1., 0., 0.]) tensor([0., 0., 0.]) tensor([0., 0., 0.])
10 tensor([0., 1., 0.]) tensor([0., 0., 0.]) tensor([0., 1., 0.]) tensor([0., 0., 0.]) tensor([0., 0., 0.])
11 tensor([0., 1., 0.]) tensor([0., 0., 0.]) tensor([0., 1., 0.]) tensor([0., 0., 0.]) tensor([0., 0., 0.])
15 tensor([0., 0., 1.]) tensor([0., 0., 0.]) tensor([0., 0., 1.]) tensor([0., 0., 0.]) tensor([0., 0., 0.])
16 tensor([0., 0., 1.]) tensor([0., 0., 0.]) tensor([0., 0., 1.]) tensor([0., 0., 0.]) tensor([0., 0., 0.])
17 tensor([0., 0., 1.]) tensor([0., 0., 0.]) tensor([0., 0., 1.]) tensor([0., 0., 0.]) tensor([0., 0., 0.])
22 tensor([ 0.,  0., -1.])

In [1786]:
rewards.sum(axis =0)

tensor([-249.3551, -293.2074,  192.1446])

In [1787]:
(rewards*(np.power(0.999, np.arange(len(rewards))   )[:,None])).sum(axis =0)

tensor([-189.3775, -200.0503,   98.1860], dtype=torch.float64)

In [1502]:
epi_list

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  19,  20,  21,
        22,  23,  24,  41,  42,  43,  44,  63,  64,  65,  66,  67,  68,
        69,  70,  71,  77,  78,  79,  80,  81,  89,  90,  91,  92,  93,
        94,  95,  96, 105, 106, 107, 108, 109, 110, 125, 126, 127, 128,
       143, 144, 145, 146, 147, 148, 168, 169, 170, 171, 172, 173, 193,
       206, 226, 227, 228, 231, 232, 233, 234, 235, 236, 250, 251, 259,
       260, 261, 262, 263, 264, 271])

In [1506]:
epi_list[19+1]-1 != 

62

In [1581]:
edi

603

In [1583]:
epi_list[-1]

604

In [1582]:
len(epi_list[:-1])

223

In [1580]:
rewards[episodes == 0]

tensor([[   0.,    0.,    0.],
        [   1.,    0.,    0.],
        [   0.,    0.,    0.],
        ...,
        [  -1.,    0.,    1.],
        [   0.,    0.,    0.],
        [-100., -100.,  100.]])

In [1578]:

rew = rewards[episodes == 0]
rew_2 = np.copy(rew)

value_copy = values[episodes == 0]
value_copy_next = np.copy(value_copy)

age = 0

gam =0.99
gam_1 = gam



values_2 = values[episodes == 0]

epi_list = np.arange(sum(episodes == 0))[current_agent[episodes == 0,0] == (age+1)]

for epi,edi in enumerate(epi_list[:-1]):
    gam_1 = gam

    v_add = rewards[epi,age]+values[edi+1]
    print(edi,rewards[epi,age],values[edi+1])
    if epi_list[epi+1] -1 != epi_list[epi]:
        for i in range(epi_list[epi]+1,epi_list[epi+1]):
            
            v_add += gam_1*rewards[i,age]
            gam_1*=gam
            print(i,"-",v_add,gam_1,rewards[i,age])
    
    values_2[edi] = v_add 
    print('values',values_2[edi],values[edi])





0 tensor(0.) tensor(0.2430)
values tensor(0.2430) tensor(0.2430)
1 tensor(1.) tensor(0.5937)
values tensor(1.5937) tensor(0.2430)
2 tensor(0.) tensor(0.5937)
values tensor(0.5937) tensor(0.5937)
3 tensor(0.) tensor(0.5937)
values tensor(0.5937) tensor(0.5937)
4 tensor(-1.) tensor(0.6397)
values tensor(-0.3603) tensor(0.5937)
5 tensor(0.) tensor(0.6397)
values tensor(0.6397) tensor(0.6397)
6 tensor(0.) tensor(0.6397)
values tensor(0.6397) tensor(0.6397)
7 tensor(0.) tensor(0.6397)
values tensor(0.6397) tensor(0.6397)
8 tensor(0.) tensor(-2.5403)
9 - tensor(-2.5403) 0.9801 tensor(0.)
10 - tensor(-2.5403) 0.9702989999999999 tensor(0.)
11 - tensor(-2.5403) 0.96059601 tensor(0.)
12 - tensor(-2.5403) 0.9509900498999999 tensor(0.)
13 - tensor(-2.5403) 0.9414801494009999 tensor(0.)
14 - tensor(-2.5403) 0.9320653479069899 tensor(0.)
15 - tensor(-2.5403) 0.92274469442792 tensor(0.)
16 - tensor(-2.5403) 0.9135172474836407 tensor(0.)
values tensor(-2.5403) tensor(0.6397)
17 tensor(0.) tensor(0.478

In [1525]:
#values_2

In [1485]:
rew = rewards[episodes == 0]
rew_2 = np.copy(rew)

value_copy = values[episodes == 0]
value_copy_next = np.copy(value_copy)

age = 1

gam =0.99
gam_1 = gam




for i in range(sum(episodes == 0)):
    
    print(current_agent[i].numpy()[0].astype(int),i, current_agent[i].numpy()[0].astype(int)==age,   )
     





1 0 True
1 1 True
1 2 True
1 3 True
1 4 True
1 5 True
1 6 True
1 7 True
1 8 True
1 9 True
2 10 False
2 11 False
3 12 False
3 13 False
3 14 False
3 15 False
3 16 False
3 17 False
3 18 False
1 19 True
1 20 True
1 21 True
1 22 True
1 23 True
1 24 True
2 25 False
2 26 False
2 27 False
2 28 False
2 29 False
2 30 False
2 31 False
3 32 False
3 33 False
3 34 False
3 35 False
3 36 False
3 37 False
3 38 False
3 39 False
3 40 False
1 41 True
1 42 True
1 43 True
1 44 True
2 45 False
2 46 False
2 47 False
2 48 False
2 49 False
3 50 False
3 51 False
3 52 False
3 53 False
3 54 False
3 55 False
3 56 False
3 57 False
3 58 False
3 59 False
3 60 False
3 61 False
3 62 False
1 63 True
1 64 True
1 65 True
1 66 True
1 67 True
1 68 True
1 69 True
1 70 True
1 71 True
2 72 False
2 73 False
2 74 False
2 75 False
3 76 False
1 77 True
1 78 True
1 79 True
1 80 True
1 81 True
2 82 False
3 83 False
3 84 False
3 85 False
3 86 False
3 87 False
3 88 False
1 89 True
1 90 True
1 91 True
1 92 True
1 93 True
1 94 True
1 95 

In [1447]:
step

22340

In [1440]:
#obs[:10]

In [1389]:
observation['action_mask']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 1], dtype=int8)

In [1343]:
action, logprob, _, value = agent_mod.get_action_and_value(model_in)
action

tensor([[19.0000,  0.4983]], grad_fn=<CatBackward0>)

In [801]:



for iteration in range(1, 5000):#args.num_iterations) :
    if args.anneal_lr:
            frac = 1.0 - (iteration - 1.0) / 10000#args.num_iterations
            lrnow = frac * lr#args.learning_rate
            optimizer.param_groups[0]["lr"] = lrnow

    step = 0
    for episode in range(num_episodes):
    
        action=1
        if fault_condition:
            env = env_risk(**(env_config | {"render_mode" : None}))#game.env(render_mode=None)
        env.reset()
        fault_condition = False
        #if step>0:
            #print( observation['action_mask'])
        for agent in env.agent_iter():
            
            observation, reward, termination, truncation, info = env.last()
        
            episodes[step] = episode + (iteration-2)*num_episodes
            current_phase[step] = phase
            
            #global_step += args.num_envs
        
            if True:#action != None: # this is when a player is removed
                
                obs[step] = torch.Tensor(observation['observation'],device=device) #sould i not add it .... meaning this is the last observation after the player dies
                action_masks[step] = torch.Tensor(observation['action_mask'],device=device)
                curr_agent = agent#int(agent[-1])
                current_agent[step] = curr_agent
                
            if step >0: # the only thing is we would not have a reward for him if the player is done.... 
                #actions[step-1] = action
                rewards[step-1] = reward
                dones[step-1] = termination or truncation
                
            if action == None:
                rewards[step] = 0 # should i keep it -1? .... hm i dont think so .
                dones[step] = 0 # frankly the guys is already done so we really dont have to do anything here.... this is the state post termination for a loser 
                # but btw this is for the next agent ... action == None means in the last action the previous agent would have been removed.
                #values[step] = 
            
            #obs[step] = next_obs
            #dones[step] = next_done
        
            #print(agent,action,termination or truncation, reward,info,observation['action_mask'],env.terminations,env.truncations)






            model_in = torch.Tensor(np.hstack((observation_step['observation'].reshape(-1),observation_step['action_mask'].reshape(-1)
                                                          ,[curr_agent]))[None,:]#.repeat(3,axis = 0)
                                    )

            
            if termination or truncation:
                action = None
                with torch.no_grad():
                    act, logprob, _, value = agent_mod.get_action_and_value(model_in)
                    values[step] = value.flatten() # so even if we are removing the guy ... we need to know what is the action he would 
                                                    #have taken at this point and what would have been its value
                actions[step] = act #even after going what would have been
                logprobs[step] = logprob        
            else:
                mask = observation["action_mask"]
                # this is where you would insert your policy
                #action = env.action_space(agent).sample(mask)
        
                # ALGO LOGIC: action logic
                with torch.no_grad():
                    #print(next_obs.shape)
                    action, logprob, _, value = agent_mod.get_action_and_value(model_in)
                    values[step] = value.flatten()
                actions[step] = action
                logprobs[step] = logprob
                
                if not observation['action_mask'][action]: 
                    fault_condition =True
                    faulting_player = agent
                    #print('here',agent, action, observation['action_mask'])
    
            #print('here',agent, action)
            step +=1
            global_step+=1
            env.step(action.numpy() if action != None else None)


            

            
            if (global_step == num_steps):# or (fault_condition and (faulting_player != agent) and (len(env.agents)==0)):
                break
    
        if global_step == num_steps:
            break
    if global_step == num_steps:
        break

#env.close()
    with torch.no_grad():
        advantages = torch.zeros_like(rewards).to(device)
    
        for i_epi in range((iteration-2)*num_episodes,episode + (iteration-2)*num_episodes):
            for i in agent_list:
                agents_id = int(i[-1])
                 
                cur_index = torch.where((current_agent[:,0] == agents_id) &( episodes == i_epi))[0]
            
            
                # bootstrap value if not done
            
                #next_value = agent.get_value(next_obs).reshape(1, -1)
                
                lastgaelam = 0
        
        
                last_index = cur_index[-1]
                nextnonterminal = 1.0
                delta = rewards[last_index] - values[last_index]
                #print(agents_id,last_index,nextnonterminal,delta,rewards[last_index],values[last_index])
                advantages[last_index] = lastgaelam = delta + args.gamma * args.gae_lambda * nextnonterminal * lastgaelam
                
                
                for o,t in reversed(list(enumerate(cur_index))[:-1]):
                    
        
                    nextnonterminal = 1.0 - dones[cur_index[o + 1]]
                    nextvalues = values[cur_index[o + 1]]
                    delta = rewards[t] + args.gamma * nextvalues * nextnonterminal - values[t]
                    #print(agents_id,o,t,cur_index[o + 1],nextnonterminal,nextvalues,delta,rewards[t])
                    advantages[t] = lastgaelam = delta + args.gamma * args.gae_lambda * nextnonterminal * lastgaelam
                returns = advantages + values
        
        
    #print(agent_list)
    
    
    
    
    
    
    
    
    b_obs = obs.reshape(obs.shape)
    b_logprobs = logprobs.reshape(-1)
    b_actions = actions.reshape(actions.shape)
    b_action_masks = action_masks.reshape(action_masks.shape)
    b_advantages = advantages.reshape(-1)
    b_returns = returns.reshape(-1)
    b_values = values.reshape(-1)
    b_obs_a = torch.concat(( b_obs.reshape(-1,np.prod(ob_space_shape)) ,b_action_masks.reshape(-1,np.prod(action_mask_shape)),current_agent ),axis =1)
    

    # Optimizing the policy and value network
    b_inds = np.arange(step)#args.batch_size)
    clipfracs = []
    for epoch in range(args.update_epochs):
        np.random.shuffle(b_inds)
        for start in range(0, step, args.minibatch_size):
            end = start + args.minibatch_size
            mb_inds = b_inds[start:end]
    
            _, newlogprob, entropy, newvalue = agent_mod.get_action_and_value(b_obs_a[mb_inds],# b_obs[mb_inds].reshape(-1,np.prod(ob_space_shape)),#
                                                                              
                                                                          b_actions.long()[mb_inds])
            logratio = newlogprob - b_logprobs[mb_inds]
            ratio = logratio.exp()
    
            with torch.no_grad():
                # calculate approx_kl http://joschu.net/blog/kl-approx.html
                old_approx_kl = (-logratio).mean()
                approx_kl = ((ratio - 1) - logratio).mean()
                clipfracs += [((ratio - 1.0).abs() > args.clip_coef).float().mean().item()]
    
            mb_advantages = b_advantages[mb_inds]
            if args.norm_adv:
                mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)
    
            # Policy loss
            pg_loss1 = -mb_advantages * ratio
            pg_loss2 = -mb_advantages * torch.clamp(ratio, 1 - args.clip_coef, 1 + args.clip_coef)
            pg_loss = torch.max(pg_loss1, pg_loss2).mean()
    
            # Value loss
            newvalue = newvalue.view(-1)
            if args.clip_vloss:
                v_loss_unclipped = (newvalue - b_returns[mb_inds]) ** 2
                v_clipped = b_values[mb_inds] + torch.clamp(
                    newvalue - b_values[mb_inds],
                    -args.clip_coef,
                    args.clip_coef,
                )
                v_loss_clipped = (v_clipped - b_returns[mb_inds]) ** 2
                v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
                v_loss = 0.5 * v_loss_max.mean()
            else:
                v_loss = 0.5 * ((newvalue - b_returns[mb_inds]) ** 2).mean()
    
            entropy_loss = entropy.mean()
            loss = pg_loss - args.ent_coef * entropy_loss + v_loss * args.vf_coef
    
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(agent_mod.parameters(), args.max_grad_norm)
            optimizer.step()
    
        if args.target_kl is not None and approx_kl > args.target_kl:
            break

    y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy()
    var_y = np.var(y_true)
    explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y

    # TRY NOT TO MODIFY: record rewards for plotting purposes
    if TB_log:  
        writer.add_scalar("charts/learning_rate", optimizer.param_groups[0]["lr"], global_step)
        writer.add_scalar("losses/value_loss", v_loss.item(), global_step)
        writer.add_scalar("losses/policy_loss", pg_loss.item(), global_step)
        writer.add_scalar("losses/entropy", entropy_loss.item(), global_step)
        writer.add_scalar("losses/old_approx_kl", old_approx_kl.item(), global_step)
        writer.add_scalar("losses/approx_kl", approx_kl.item(), global_step)
        writer.add_scalar("losses/clipfrac", np.mean(clipfracs), global_step)
        writer.add_scalar("losses/explained_variance", explained_var, global_step)
    if global_step%100 ==0:
        print("SPS:", int(global_step / (time.time() - start_time)))
    if TB_log:  
        writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)

env.close()
writer.close()
#return agent

[WARNING]: Illegal move made, game terminating with current player losing. 
obs['action_mask'] contains a mask of all legal moves that can be chosen.
[WARNING]: Illegal move made, game terminating with current player losing. 
obs['action_mask'] contains a mask of all legal moves that can be chosen.
[WARNING]: Illegal move made, game terminating with current player losing. 
obs['action_mask'] contains a mask of all legal moves that can be chosen.
[WARNING]: Illegal move made, game terminating with current player losing. 
obs['action_mask'] contains a mask of all legal moves that can be chosen.
[WARNING]: Illegal move made, game terminating with current player losing. 
obs['action_mask'] contains a mask of all legal moves that can be chosen.
[WARNING]: Illegal move made, game terminating with current player losing. 
obs['action_mask'] contains a mask of all legal moves that can be chosen.
[WARNING]: Illegal move made, game terminating with current player losing. 
obs['action_mask'] conta

/var/folders/vl/9q_38sj961n1slqfmcpfjgnc0000gn/T/ipykernel_22511/208731411.py:277: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1711438669293/work/aten/src/ATen/native/ReduceOps.cpp:1807.)
  mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)


ValueError: Expected parameter logits (Tensor of shape (32, 7)) of distribution Categorical(logits: torch.Size([32, 7])) to satisfy the constraint IndependentConstraint(Real(), 1), but found invalid values:
tensor([[nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan,

In [797]:
env.observation_space(playe_r).n

4

In [688]:
b_obs[mb_inds].shape

torch.Size([32, 3, 3, 2])

In [597]:
env.reset()
env.agents

['player_1', 'player_2']

In [544]:
hd = agent_mod.network(b_obs_a[mb_inds].to(dtype=torch.float32))
                                                                              
                                        #                                  b_actions.long()[mb_inds])

In [549]:
agent_mod.network(b_obs_a[None,10])

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       grad_fn=<TanhBackward0>)

In [653]:
x = torch.Tensor(np.concatenate((observation['observation'].reshape(-1),observation['action_mask'].reshape(-1) )))



hidden = agent_mod.network(x)
#logits = self.actor(hidden)

In [655]:
agent_mod.actor(hidden)


tensor([-0.0007,  0.0032, -0.0028, -0.0049, -0.0025, -0.0025,  0.0030,  0.0015,
         0.0021], grad_fn=<ViewBackward0>)

In [656]:
probs = Categorical(logits=agent_mod.actor(hidden))
#if action is None:
action = probs.sample()
#return action, probs.log_prob(action),

In [664]:
[probs.log_prob(torch.tensor(i)) for i in [0,2,3,4,5,6,7,8]]

[tensor(-2.1976, grad_fn=<SqueezeBackward1>),
 tensor(-2.1996, grad_fn=<SqueezeBackward1>),
 tensor(-2.2017, grad_fn=<SqueezeBackward1>),
 tensor(-2.1993, grad_fn=<SqueezeBackward1>),
 tensor(-2.1993, grad_fn=<SqueezeBackward1>),
 tensor(-2.1938, grad_fn=<SqueezeBackward1>),
 tensor(-2.1953, grad_fn=<SqueezeBackward1>),
 tensor(-2.1947, grad_fn=<SqueezeBackward1>)]

In [781]:

env = game.env(render_mode="human")
env.reset()



for agent in env.agent_iter():
    
    observation, reward, termination, truncation, info = env.last()

    
    if termination or truncation:
        action = None
        with torch.no_grad():
            act, logprob, _, value = agent_mod.get_action_and_value(torch.Tensor(
                
                                                                        np.concatenate((observation['observation'].reshape(-1),
                                                                                        observation['action_mask'].reshape(-1) ,
                                                                                       [int(agent[-1])]))))
            print(logprob) # so even if we are removing the guy ... we need to know what is the action he would 
                                            #have taken at this point and what would have been its value  
    else:
        mask = observation["action_mask"]

        # ALGO LOGIC: action logic
        with torch.no_grad():
            #print(next_obs.shape)
            action, logprob, _, value = agent_mod.get_action_and_value(torch.Tensor(
                
                                                                        np.concatenate((observation['observation'].reshape(-1),
                                                                                        mask.reshape(-1),[int(agent[-1])] 
                                                                                       ))))
            #values[step] = value.flatten()
        #actions[step] = action
        #logprobs[step] = logprob
        print(agent_mod.get_valid_action(torch.Tensor(
                
                                                                        np.concatenate((observation['observation'].reshape(-1),
                                                                                        mask.reshape(-1),[int(agent[-1])] 
                                                                                       )))),  agent)
    env.step(action.numpy() if action != None else None)


tensor([0.1467, 0.1517, 0.1433, 0.1264, 0.1377, 0.1450, 0.1492],
       grad_fn=<SoftmaxBackward0>) player_0
tensor([0.1463, 0.1535, 0.1435, 0.1258, 0.1375, 0.1452, 0.1482],
       grad_fn=<SoftmaxBackward0>) player_1
tensor([0.1466, 0.1527, 0.1430, 0.1263, 0.1375, 0.1451, 0.1488],
       grad_fn=<SoftmaxBackward0>) player_0
tensor([0.1463, 0.1534, 0.1432, 0.1260, 0.1374, 0.1451, 0.1485],
       grad_fn=<SoftmaxBackward0>) player_1
tensor([0.1463, 0.1532, 0.1432, 0.1261, 0.1374, 0.1451, 0.1486],
       grad_fn=<SoftmaxBackward0>) player_0
tensor([0.1463, 0.1534, 0.1432, 0.1260, 0.1374, 0.1451, 0.1485],
       grad_fn=<SoftmaxBackward0>) player_1
tensor([0.1463, 0.1534, 0.1432, 0.1260, 0.1374, 0.1451, 0.1485],
       grad_fn=<SoftmaxBackward0>) player_0
tensor([0.1463, 0.1534, 0.1432, 0.1260, 0.1374, 0.1451, 0.1485],
       grad_fn=<SoftmaxBackward0>) player_1
tensor([0.1463, 0.1534, 0.1432, 0.1260, 0.1374, 0.1451, 0.1485],
       grad_fn=<SoftmaxBackward0>) player_0
tensor([0.1463, 0.1

In [228]:
episode + (iteration-1)*num_episodes

19

In [481]:
args.minibatch_size

32

In [488]:
values.shape

torch.Size([1000])

In [496]:
    with torch.no_grad():
        advantages = torch.zeros_like(rewards).to(device)
    
        for i_epi in range(episode + (iteration-1)*num_episodes):
            for i in agent_list:
                agents_id = int(i[-1])
                 
                cur_index = torch.where((current_agent[:,0] == agents_id) &( episodes == i_epi))[0]
            
            
                # bootstrap value if not done
            
                #next_value = agent.get_value(next_obs).reshape(1, -1)
                
                lastgaelam = 0
        
        
                last_index = cur_index[-1]
                nextnonterminal = 1.0
                delta = rewards[last_index] - values[last_index]
                #print(agents_id,last_index,nextnonterminal,delta,rewards[last_index],values[last_index])
                advantages[last_index] = lastgaelam = delta + args.gamma * args.gae_lambda * nextnonterminal * lastgaelam
                
                
                for o,t in reversed(list(enumerate(cur_index))[:-1]):
                    
        
                    nextnonterminal = 1.0 - dones[cur_index[o + 1]]
                    nextvalues = values[cur_index[o + 1]]
                    delta = rewards[t] + args.gamma * nextvalues * nextnonterminal - values[t]
                    #print(agents_id,o,t,cur_index[o + 1],nextnonterminal,nextvalues,delta,rewards[t])
                    advantages[t] = lastgaelam = delta + args.gamma * args.gae_lambda * nextnonterminal * lastgaelam
                returns = advantages + values
        
        
    #print(agent_list)
    
    
    
    
    
    
    
    
    b_obs = obs.reshape(obs.shape)
    b_logprobs = logprobs.reshape(-1)
    b_actions = actions.reshape(actions.shape)
    b_action_masks = action_masks.reshape(action_masks.shape)
    b_advantages = advantages.reshape(-1)
    b_returns = returns.reshape(-1)
    b_values = values.reshape(-1)
    b_obs_a = torch.concat(( b_obs.reshape(-1,84) ,b_action_masks.reshape(-1,7)),axis =1)
    
    
    # Optimizing the policy and value network
    b_inds = np.arange(step)#args.batch_size)
    clipfracs = []
    for epoch in range(1):#args.update_epochs):
        np.random.shuffle(b_inds)
        for start in range(0, step, args.minibatch_size):
            end = start + args.minibatch_size
            mb_inds = b_inds[start:end]
    
            _, newlogprob, entropy, newvalue = agent_mod.get_action_and_value(b_obs_a[mb_inds], 
                                                                              
                                                                          b_actions.long()[mb_inds])
            logratio = newlogprob - b_logprobs[mb_inds]
            ratio = logratio.exp()
    
            with torch.no_grad():
                # calculate approx_kl http://joschu.net/blog/kl-approx.html
                old_approx_kl = (-logratio).mean()
                approx_kl = ((ratio - 1) - logratio).mean()
                clipfracs += [((ratio - 1.0).abs() > args.clip_coef).float().mean().item()]
    
            mb_advantages = b_advantages[mb_inds]
            if args.norm_adv:
                mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)
    
            # Policy loss
            pg_loss1 = -mb_advantages * ratio
            pg_loss2 = -mb_advantages * torch.clamp(ratio, 1 - args.clip_coef, 1 + args.clip_coef)
            pg_loss = torch.max(pg_loss1, pg_loss2).mean()
    
            # Value loss
            newvalue = newvalue.view(-1)
            if args.clip_vloss:
                v_loss_unclipped = (newvalue - b_returns[mb_inds]) ** 2
                v_clipped = b_values[mb_inds] + torch.clamp(
                    newvalue - b_values[mb_inds],
                    -args.clip_coef,
                    args.clip_coef,
                )
                v_loss_clipped = (v_clipped - b_returns[mb_inds]) ** 2
                v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
                v_loss = 0.5 * v_loss_max.mean()
            else:
                v_loss = 0.5 * ((newvalue - b_returns[mb_inds]) ** 2).mean()
    
            entropy_loss = entropy.mean()
            loss = pg_loss - args.ent_coef * entropy_loss + v_loss * args.vf_coef
    
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(agent_mod.parameters(), args.max_grad_norm)
            optimizer.step()
    
        if args.target_kl is not None and approx_kl > args.target_kl:
            break

In [178]:
current_agent[torch.where(current_agent == 0)[0]]

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [166]:
env = connect_four_v3.env(render_mode="human")
env.reset()
[i for i in env.agents]


['player_0', 'player_1']

In [173]:
returns[:30],advantages[:30], values[:30]

(tensor([ 0.0000, -0.0301, -0.0376, -0.0278, -0.0333, -0.0227, -0.0210, -0.0181,
         -0.0184, -0.0121, -0.0172, -0.0083, -0.0108, -0.0084, -0.0077, -0.0048,
          0.0000,  0.0000, -0.6175,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]),
 tensor([ 0.0000, -0.0488,  0.0353,  0.1125,  0.2168,  0.2567,  0.6146,  0.2333,
          0.1320,  0.2982,  0.0595,  0.1919,  0.3194, -0.0020,  0.1568,  0.1759,
          0.3862,  0.2427,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]),
 tensor([ 0.0000,  0.0188, -0.0729, -0.1403, -0.2501, -0.2794, -0.6356, -0.2514,
         -0.1504, -0.3103, -0.0767, -0.2002, -0.3302, -0.0064, -0.1645, -0.1807,
         -0.3862, -0.2427, -0.6175,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]))

In [154]:
with torch.no_grad():
    advantages = torch.zeros_like(rewards).to(device)
    
    for i in [0,1]:
        agents_id = i#int(i[-1])
         
        cur_index = torch.where(current_agent == agents_id)[0]
    
        print(cur_index)
        # bootstrap value if not done
    
        #next_value = agent.get_value(next_obs).reshape(1, -1)
        
        lastgaelam = 0

        
        
        for o,t in reversed(list(enumerate(cur_index))[:-1]):
            

            
            nextnonterminal = 1.0 - dones[cur_index[o + 1]]
            nextvalues = values[cur_index[o + 1]]
            delta = rewards[t] + args.gamma * nextvalues * nextnonterminal - values[t]

            print(agents_id,nextnonterminal,nextvalues,delta)
            
            advantages[t] = lastgaelam = delta + args.gamma * args.gae_lambda * nextnonterminal * lastgaelam
        returns = advantages + values

tensor([ 2,  4,  6,  8, 10, 12, 14, 16, 18])
0 tensor(0.) tensor(-0.6175) tensor(0.3862)
0 tensor(1.) tensor(-0.3862) tensor(-0.2213)
0 tensor(1.) tensor(-0.1645) tensor(0.1659)
0 tensor(1.) tensor(-0.3302) tensor(-0.2532)
0 tensor(1.) tensor(-0.0767) tensor(0.0738)
0 tensor(1.) tensor(-0.1504) tensor(0.4854)
0 tensor(1.) tensor(-0.6356) tensor(-0.3849)
0 tensor(1.) tensor(-0.2501) tensor(-0.1770)
tensor([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19])
1 tensor(0.) tensor(0.) tensor(0.2427)
1 tensor(1.) tensor(-0.2427) tensor(-0.0617)
1 tensor(1.) tensor(-0.1807) tensor(-0.1742)
1 tensor(1.) tensor(-0.0064) tensor(0.1939)
1 tensor(1.) tensor(-0.2002) tensor(0.1103)
1 tensor(1.) tensor(-0.3103) tensor(-0.0586)
1 tensor(1.) tensor(-0.2514) tensor(0.0282)
1 tensor(1.) tensor(-0.2794) tensor(-0.1388)
1 tensor(1.) tensor(-0.1403) tensor(-0.1590)


In [155]:
advantages[:30]

tensor([ 0.0000, -0.0488,  0.0353,  0.1125,  0.2168,  0.2567,  0.6146,  0.2333,
         0.1320,  0.2982,  0.0595,  0.1919,  0.3194, -0.0020,  0.1568,  0.1759,
         0.3862,  0.2427,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000])

In [152]:
[i for i in env.agents]

[]

In [75]:
torch.where(current_agent == np.array([1,]))[0]

tensor([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21])

In [107]:
ob_space_shape

(6, 7, 2)

In [81]:
torch.where(current_agent == 1)[0]

tensor([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21])

In [92]:
for i,j in reversed(list(enumerate([5,2,3]))):
    print(i,j)

2 3
1 2
0 5


In [95]:
list(enumerate([5,2,3]))[1:]

[(1, 2), (2, 3)]

In [101]:
observation['observation']

array([[[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]]], dtype=int8)